# ELVTR Data Science Main Project

## Deliverables

01 Git Repository

Include all project code with a README file containing a high-level project description.

Example README guide: [Make a README](link-to-readme-guide)

Report

* Methodology, approach, and model selection rationale.
* Advantages and limitations of the chosen model.
* Architecture of the final solution.
* Considerations on deployment and scalability of the solution - i.e., how will the model be used in BAU by the business?
* Estimated impact/ROI of the project.

# Data Science in Finance: Lending Club Loan Analysis

## Project Overview

Lending Club has tasked us with preparing a loan application dataset for analysis and predictive modeling. 

The key tasks include data cleaning, exploratory data analysis, and building a predictive model for loan classification. An optional component involves building a real-time scoring application.

**Project Objectives**:
- Clean and preprocess the data.
- Perform exploratory data analysis (EDA) to gain insights.
- Develop a predictive model for loan application approval.
- (Optional) Build a real-time scoring application.

**Dataset Description**:
The dataset consists of loan application records, including various financial metrics and the application status. The data dictionary is provided for understanding the attributes.

**Dataset Path**:
- CSV: `data/1-raw/lending-club-2007-2020Q3/Loan_status_2007-2020Q3-100ksample.csv`
- Data Dictionary: `data/1-raw/lending-club-2007-2020Q3/LCDataDictionary.xlsx`

In [1]:
import pandas as pd

In [2]:
pwd

'c:\\Users\\kiera\\OneDrive\\Documents\\GitHub\\dsif-git-main-project\\elvtr_main_project\\notebooks'

## Load Data Set

In [ ]:
# Load the data xlsx file as a dataframe
df = pd.read_csv("c:\\Users\\kiera\\OneDrive\\Documents\\GitHub\\dsif-git-main-project\\elvtr_main_project\\data\\raw\\Loan_status_2007-2020Q3\\Loan_status_2007-2020Q3-100k-Full-Data.csv")

# Clean headers in the existing DataFrame 'df'
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

# Display cleaned headers
print("Cleaned headers:", df.columns.tolist())

df.shape

In [ ]:
df.head()

In [ ]:
# Load the data dictionary CSV file as a dataframe
df_data_dict = pd.read_excel("c:\\Users\\kiera\\OneDrive\\Documents\\GitHub\\dsif-git-main-project\\elvtr_main_project\\data\\raw\\Loan_status_2007-2020Q3\\LCDataDictionary.xlsx")

# Clean headers in the existing DataFrame 'df'
df_data_dict.columns = df_data_dict.columns.str.strip().str.lower().str.replace(" ", "_")

# Remove trailing whitespaces in all string columns of df_data_dict
df_data_dict = df_data_dict.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

# Display cleaned headers
print("Cleaned headers:", df_data_dict.columns.tolist())

df_data_dict.shape

In [ ]:
# Copies the columns and descriptions from the data dictionary into a data frame for future recall.
# Initialize empty lists for LoanStatNew and Description
loanstatnew = []
description = []

# Iterate through each row in the DataFrame and populate lists
for _, row in df_data_dict.iterrows():
    loanstatnew.append(row['loanstatnew'])
    description.append(row['description'])

# Apply left-aligned styling to both headers and data cells
styled_df_data_dict = df_data_dict.style.set_properties(
    **{'text-align': 'left', 'white-space': 'nowrap'}
).set_table_styles(
    [{'selector': 'th', 'props': [('text-align', 'left')]}]
)

# Display styled DataFrame
styled_df_data_dict

After analysing our data dictionary it is possible to class our columns into figurative categories to better organise our analysis. 

These categories are, for now:

- Credit history, 
- Current Debt and Payment behaviours, 
- Employement, 
- Credit inquiries
- Loan Application information
- Hardship and Settlement Information
- Co-Borrower Information
- Loan Performance

Let's create a table for reference. We'll add these manually so that we can tweak the data within each group as we discover more about our data.

## Display Basic Data Discovery

Let's look at the information within our data frame (df) looking at our initial feature set (pre_hardship_fields).

In [ ]:
df.head()

Our data contains 143 columns and 99999 rows of data. It is comprised of numerical (float, int) and categorical data (object)

We've ran into an error (KeyError: "['desc', 'member_id', 'verified_status_joint'] not in index"), for the time being we'll remove the missing fields from df_data_dict. These are most likely naming issues i.e. member_id is most likely id but considering we won't be using this data for now we can remove it and correct later on.

In [ ]:
# Extract column headers from df and the first column of df_data_dict
df_columns = set(df.columns)
df_data_dict_columns = set(df_data_dict.iloc[:, 0])  # First column of df_data_dict

# Find columns in df that are missing in df_data_dict
missing_in_data_dict = df_columns - df_data_dict_columns

# Find columns in df_data_dict that are missing in df
missing_in_df = df_data_dict_columns - df_columns

# Output the results
print("Columns in df that are missing in df_data_dict:")
print(missing_in_data_dict)

print("\nColumns in df_data_dict that are missing in df:")
print(missing_in_df)

Let's filter the values that are missing in df.

In [ ]:
# This list was created to run analysis later on. I've opted to select the pre_hardship_fields as my feature selection draft
# Subjective groupings created for post ML analysis. Due to the transformation these lists can only be used with df
groups = {
    "Credit History": [
        'earliest_cr_line', 'fico_range_high', 'fico_range_low', 'last_fico_range_high',
        'last_fico_range_low', 'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op', 'num_accts_ever_120_pd',
        'num_tl_120dpd_2m', 'pub_rec', 'pub_rec_bankruptcies'
    ],
    "Current Debt and Payment Behaviors": [
        'acc_now_delinq', 'all_util', 'bc_open_to_buy', 'bc_util', 'chargeoff_within_12_mths',
        'collections_12_mths_ex_med', 'delinq_2yrs', 'delinq_amnt', 'max_bal_bc', 'mths_since_last_delinq',
        'num_rev_accts', 'num_rev_tl_bal_gt_0', 'percent_bc_gt_75', 'revol_bal', 'revol_util',
        'tot_coll_amt', 'tot_cur_bal', 'tot_hi_cred_lim', 'total_bal_ex_mort', 'total_bc_limit'
    ],
    "Employment": [
        'emp_length', 'emp_title', 'annual_inc', 'annual_inc_joint'
    ],
    "Credit Inquiries": [
        'inq_fi', 'inq_last_12m', 'inq_last_6mths', 'num_tl_op_past_12m'
    ],
    "Loan Application Information": [
        'loan_amnt', 'term', 'int_rate', 'application_type', 'grade', 'sub_grade', 'purpose',
        'issue_d', 'home_ownership', 'zip_code', 'addr_state', 'title', 'desc', 'url'
    ],
    "Hardship and Settlement Information": [
        'hardship_flag', 'hardship_type', 'hardship_reason', 'hardship_status', 'hardship_start_date',
        'hardship_end_date', 'hardship_amount', 'hardship_length', 'settlement_status', 'settlement_date',
        'settlement_amount', 'settlement_percentage', 'settlement_term'
    ],
    "Co-Borrower Information": [
        'annual_inc_joint', 'dti_joint', 'verified_status_joint', 'revol_bal_joint',
        'sec_app_fico_range_low', 'sec_app_fico_range_high', 'sec_app_earliest_cr_line',
        'sec_app_mort_acc', 'sec_app_open_acc', 'sec_app_revol_util'
    ],
    "Loan Performance": [
        'funded_amnt', 'funded_amnt_inv', 'out_prncp', 'out_prncp_inv', 'total_pymnt',
        'total_pymnt_inv', 'total_rec_int', 'total_rec_late_fee', 'total_rec_prncp', 'recoveries',
        'collection_recovery_fee', 'last_pymnt_amnt', 'last_pymnt_d', 'next_pymnt_d'
    ]
}

# Define pre and post hardship fields
pre_hardship_fields = [
    'acc_now_delinq', 'acc_open_past_24mths', 'addr_state', 'all_util', 'annual_inc', 
    'annual_inc_joint', 'application_type', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util', 
    'chargeoff_within_12_mths', 'collections_12_mths_ex_med', 'delinq_2yrs', 'delinq_amnt', 
    'dti', 'dti_joint', 'earliest_cr_line', 'emp_length', 'emp_title', 
    'fico_range_high', 'fico_range_low', 'funded_amnt', 'funded_amnt_inv', 'grade', 
    'home_ownership', 'il_util', 'initial_list_status', 'inq_fi', 'inq_last_12m', 
    'inq_last_6mths', 'installment', 'int_rate', 'issue_d', 'loan_amnt', 'loan_status', 
    'max_bal_bc', 'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op', 
    'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mort_acc', 'mths_since_last_delinq', 
    'mths_since_last_major_derog', 'mths_since_last_record', 'mths_since_rcnt_il', 
    'mths_since_recent_bc', 'mths_since_recent_bc_dlq', 'mths_since_recent_inq', 
    'mths_since_recent_revol_delinq', 'num_accts_ever_120_pd', 'num_actv_bc_tl', 
    'num_actv_rev_tl', 'num_bc_sats', 'num_bc_tl', 'num_il_tl', 'num_op_rev_tl', 
    'num_rev_accts', 'num_rev_tl_bal_gt_0', 'num_sats', 'num_tl_120dpd_2m', 'num_tl_30dpd', 
    'num_tl_90g_dpd_24m', 'num_tl_op_past_12m', 'open_acc', 'open_acc_6m', 'open_il_12m', 
    'open_il_24m', 'open_act_il', 'open_rv_12m', 'open_rv_24m', 'out_prncp', 
    'out_prncp_inv', 'pct_tl_nvr_dlq', 'percent_bc_gt_75', 'policy_code', 'pub_rec', 
    'pub_rec_bankruptcies', 'purpose', 'pymnt_plan', 'revol_bal', 'revol_util', 
    'sub_grade', 'tax_liens', 'term', 'title', 'tot_coll_amt', 'tot_cur_bal', 
    'tot_hi_cred_lim', 'total_acc', 'total_bal_ex_mort', 'total_bal_il', 'total_bc_limit', 
    'total_cu_tl', 'total_il_high_credit_limit', 'total_pymnt', 'total_pymnt_inv', 
    'total_rec_int', 'total_rec_late_fee', 'total_rec_prncp', 'total_rev_hi_lim', 
    'verification_status', 'zip_code'
]

post_hardship_fields = [
    'hardship_flag', 'hardship_type', 'hardship_reason', 'hardship_status', 
    'hardship_start_date', 'hardship_end_date', 'hardship_amount', 'hardship_length', 
    'hardship_dpd', 'hardship_loan_status', 'orig_projected_additional_accrued_interest', 
    'hardship_payoff_balance_amount', 'hardship_last_payment_amount', 'disbursement_method', 
    'debt_settlement_flag', 'debt_settlement_flag_date', 'settlement_status', 'settlement_date', 
    'settlement_amount', 'settlement_percentage', 'settlement_term'
]

In [ ]:
# Filter pre_hardship_fields to only include values that exist as columns in df
pre_hardship_fields_clean_kn = [field for field in pre_hardship_fields if field in df.columns]

# Display the filtered list
print("Filtered pre_hardship_fields:", pre_hardship_fields_clean_kn)

In [ ]:
df[pre_hardship_fields_clean_kn].head()

In [ ]:
df[pre_hardship_fields_clean_kn].describe()

In [ ]:
# Initialize list to collect cross-tab data
cross_tab_data = []

# Iterate over each group and each field to determine availability
for group, fields in groups.items():
    for field in fields:
        pre = 'Yes' if field in pre_hardship_fields else 'No'
        post = 'Yes' if field in post_hardship_fields else 'No'
        cross_tab_data.append([group, field, pre, post])

# Create DataFrame for cross-tab
cross_tab_df = pd.DataFrame(cross_tab_data, columns=["Group", "Field", "Pre-Hardship", "Post-Hardship"])

# Apply styling to left-align specific columns and set table header alignment
cross_tab_df_styled = cross_tab_df.style.set_properties(
    subset=['Group', 'Field'],
    **{'text-align': 'left'}
).set_table_styles(
    [{'selector': 'th.col_heading.level0', 'props': [('text-align', 'left')]}]
)

# Display the styled DataFrame
cross_tab_df_styled

Because are scoring is supposed to identify good and bad payers we'll leverage Pre-Hardship feature list for further analysis.

In [ ]:
#load the employtment mapping CSV file as a dataframe
df_emp_title = pd.read_csv("c:\\Users\\kiera\\OneDrive\\Documents\\GitHub\\dsif-git-main-project\\elvtr_main_project\\data\\raw\\emp_title_mapping.csv")

# Clean headers in the existing DataFrame
df_emp_title.columns = df_emp_title.columns.str.strip().str.lower().str.replace(" ", "_")

# Display cleaned headers
print("Cleaned headers:", df_emp_title.columns.tolist())

df_emp_title.shape

In [ ]:
# Initialize empty lists for LoanStatNew and Description
jobtitle = []
job_category = []

# Iterate through each row in the DataFrame and populate lists
for _, row in df_emp_title.iterrows():
    jobtitle.append(row['job_title'])
    job_category.append(row['category'])

# Apply left-aligned styling to both headers and data cells
styled_df_emp_title = df_emp_title.style.set_properties(
    **{'text-align': 'left', 'white-space': 'nowrap'}
).set_table_styles(
    [{'selector': 'th', 'props': [('text-align', 'left')]}]
)

# Display styled DataFrame
styled_df_emp_title

In [ ]:
# Cross table on 'category' in df_emp_title
category_crosstab = pd.crosstab(index=df_emp_title['category'], columns='count').sort_values(by='count', ascending=False)

# Display the crosstab
category_crosstab

Let's import our libraries and configure any paramaters for charting later on.

In [ ]:
# Essential libraries for data manipulation, statistics, and visualization
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import normaltest, shapiro, anderson, kstest, skew

# Visualization libraries
import matplotlib.pyplot as plt  # For standard plotting
import seaborn as sns  # For static plots with themes
import plotly.express as px  # For interactive plots
import missingno as msno  # For missing data visualization

# Machine Learning libraries
from sklearn.preprocessing import StandardScaler, LabelEncoder, label_binarize
from sklearn.model_selection import train_test_split, cross_val_score  # Data splitting and cross-validation
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import (
    RandomForestClassifier, RandomForestRegressor,
    GradientBoostingClassifier, GradientBoostingRegressor
)
from sklearn.svm import SVC, SVR  # Support Vector Machines for classification and regression
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.naive_bayes import GaussianNB  # Naive Bayes Classifier
from sklearn.cluster import KMeans  # K-Means clustering
from sklearn.decomposition import PCA  # Dimensionality reduction
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier
from sklearn.feature_selection import RFE  # Recursive Feature Elimination

# Additional machine learning models
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Evaluation metrics
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, roc_curve, precision_recall_curve, average_precision_score
)

# Utility libraries
from tqdm import tqdm
import joblib

# Pandas display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Plot settings for consistent figure size (A4 landscape top half)
FIG_WIDTH = 11.69  # Width
FIG_HEIGHT = 4.14  # Height

# Set the theme for Seaborn plots
sns.set_theme(style='whitegrid')

### Basic Data Overview

Before performing any analysis, we will explore the structure of the dataset to understand the nature of the available data. This includes checking the number of rows and columns, the data types of each feature, and identifying any missing values. Understanding these characteristics is essential for guiding data cleaning and feature engineering steps later in the analysis.

In [ ]:
df[pre_hardship_fields].info()

We can see that our data frame contains 100k rows, 104 columns.

Our Data set contains int, float, and string objects.

Let's check some basic statistics against all int and float data in our data frame.

In [ ]:
# Get basic statistics only for int and float columns
df[pre_hardship_fields].describe(include=['int', 'float'])

We can ignore the id columns, and will drop these as part of our basic data cleaning.

### High level analysis

Scrolling from left to right we can make the following observations:

- The Standard deviation for `annual_inc` is > 87k suggesting large disparity in numbers (we'll check this later on when pulling our distribution plots)
- The average `open_acc` is equal to 11 with a high of up to 86. For UK standards this can be considered extremely high. Worth taking this into account as a feature for our deeper analysis.
- Our delinquency fields show that we have a low average in the `delinq_2yrs` column and an average of 35 months since the last delinquency (`mths_since_last_delinq`) these could be a great indicators. 

Althought there are more lets continue our analysis and feature selection for our machine learning excercise.

## Missing Data

### Checking for Missing Values
To ensure data integrity, we check for missing values in the dataset. The `isnull()` function is used to identify null entries, and the results are sorted by the number of missing values per column. This provides insight into the columns with the most missing data, which could impact our analysis and model performance.


In [ ]:
# Check for missing values in each column
missing_values = df[pre_hardship_fields].isnull().sum().sort_values(ascending=False)
print(f"There is a total of: {len(missing_values)} columns that are missing data\n")
# print("\nMissing values in each column:\n") 
# print(missing_values[missing_values > 90000]) # Display only columns with missing values
missing_values

### Missing Data Strategy
Instead of dropping columns with a high number of missing values, we may want to retain them for our analysis. Following the logic of Abraham Wald's famous airplane and bullet holes approach, it could be beneficial to analyse the data we don't have rather than discard potentially useful columns. 

This is especially relevant for improving loan default predictions as the absence of data in itself is indicative of the possible risk to default on loans.

### Confirming DataFrame Shape

After exploring the missing values, we validate the shape of our dataframe to ensure that the dataset remains unchanged. 

This step helps confirm that we are still working with the full set of features and rows taking into account we've already stripped the first 3 columns.


In [ ]:
# Print the shape of the full DataFrame
print("Shape of the full DataFrame df:", df.shape)

# Print the shape of the DataFrame subset with pre_hardship_fields columns
print("Shape of df with only pre_hardship_fields columns:", df[pre_hardship_fields].shape)

# Print the number of columns in pre_hardship_fields
print("Number of columns in pre_hardship_fields:", len(pre_hardship_fields))


Next, we'll use the `missingno` library to analyze the missing data. This will help us understand how missing data is distributed across the dataset and the correlation between missing values in different columns.

The `missing_values` variable has been defined earlier in our workflow to quantify the total number of missing entries in each column. 

Now, we'll leverage this list to visualize the missing data using  missingno.

In [ ]:
# Filter for columns with missing values greater than 0
missing_values_graph = df[missing_values[missing_values > 0].index]

print("Categorical Data Missing Values\n")

# Visualize the missing data using the missingno library
msno.matrix(missing_values_graph)
msno.bar(missing_values_graph)
msno.heatmap(missing_values_graph)
# msno.dendrogram(missing_values_graph) #removed for the final anlysis to avoid cluttering the document with the same data but a different way to show it

### Summary of our missing data findings

#### 1. Missing Data Matrix Plot:
- The matrix plot visualizes the distribution of missing data across the dataset columns.
- Some columns have no missing data, but a few are significantly affected.
- Several columns have a substantial percentage of missing data, some exceeding 75%.
- Key columns with high missing data include:
  - `hardship_*` related fields, `payment_plan_start_date`, `sec_app_*` fields, etc.

#### 2. Missing Data Heatmap:
- The heatmap shows correlations between columns with missing data.
- Higher intensity colors indicate stronger correlations.
- Examples include `emp_title`, `mths_since_last_major_derog`, and `revol_util` showing linked missingness.
- Strong correlations exist between certain groups of columns, suggesting shared patterns in their missing values.
- We can clearly distinguish 4 groups. The three largest relating to `hardship_`, `sec_`, and `Acc` data.

### Creating Missing Value Indicators

Opposed to solomly removing values and using collected data to predict loan defaults I will create indicator variables that flag whether a value was missing for a given feature. 

This allows us to retain missing values while also capturing information about whether a data point was reported or not, which could enhance our analysis.

In [ ]:
# Creating a new list by selecting specific groups from the logical mapping we created earlier
selected_groups = ["Credit History", "Employment", "Credit Inquiries"] # created after reading the data dictionary
missing_value_indicator = sum([groups[group] for group in selected_groups], [])

In [ ]:
print(len(missing_value_indicator)) # collection of columns from our grouping whilst reading the data dictionary
print(len(pre_hardship_fields)) # the columns that are most likely pre any loan defaults.

Our draft list of features (`draft_features`) now includes 122 columns identified during the analysis of the missing data, and the basic statistics from `df` along with the information within the data dictionary.

In [ ]:
# Selection of initial features based on our previous findings and the descriptions within the data dictionary
new_features = []
new_features.extend(pre_hardship_fields)

Let's expand `df` with new columns to keep a record of the missingness values.

In [ ]:
# Empty list to store the new column names
new_missing_columns = []

# Iterating through the list to create missing value indicator columns
for col in missing_value_indicator:
    indicator_col_name = f"{col}_missing_clean_kn"  # Create a new column name for the missing indicator
    df[indicator_col_name] = df[col].isnull().astype(int)  # 1 for missing, 0 for not missing
    
    # Append the new column name to the list
    new_missing_columns.append(indicator_col_name)

# Display the list of new column names
print("New missing indicator columns:", new_missing_columns)
print("Count of New Missing Columns created", len(new_missing_columns))

In [ ]:
# Example to verify the new columns
df[new_missing_columns].head()  # Display the first few rows to verify

In [ ]:
# Append the new items to the existing list
new_features.extend(new_missing_columns)
len(new_features)

Let's check for uniqueness in our feature list.

Here we're looking for variability i.e. which features contain the most variability. The below code will provide us with a list of unique value counts for each feature within our data set, ordered in decending order, and with a % that reflects the uniqueness i.e. if we have 100k unique values (id field) then the % uniqueness will be 100%. This little variability will not add value in our analysis.

In [ ]:
def analyze_uniqueness(df):
    """
    Analyzes the number of unique values in all columns within a DataFrame, 
    regardless of their data type, and helps identify suitable candidates for visualization.

    Args:
        df: The Pandas DataFrame to analyze.

    Returns:
        A DataFrame with columns, unique value counts, and percentages.
        Also, displays a bar graph for the unique counts sorted from highest to lowest.
    """
    total_rows = len(df)

    # Calculate unique counts and percentages and store them in a DataFrame
    uniqueness_df = pd.DataFrame({
        "Unique Count": df.nunique(),
        "Unique Percentage": df.nunique() / total_rows * 100
    }).sort_values(by="Unique Count", ascending=False)

    # Print sorted output with unique counts and percentages
    print(uniqueness_df)

    # Create a bar graph to visualize the number of unique values
    plt.figure(figsize=(12, 6))
    plt.bar(uniqueness_df.index, uniqueness_df["Unique Count"])
    plt.xlabel("Columns", fontsize=14)
    plt.ylabel("Number of Unique Values", fontsize=14)
    plt.title("Unique Value Counts per Column (Sorted)", fontsize=16)
    plt.xticks(rotation=45, ha="right", fontsize=10)  # Rotate x-axis labels for readability
    plt.tight_layout()
    plt.show()

    return uniqueness_df

# Example usage with pre-selected columns
# Assuming df and pre_hardship_fields are defined and valid
#try:
#    unique_counts_df = analyze_uniqueness(df[pre_hardship_fields])
#except ValueError as e:
#    print(f"Error: {e}")

In [ ]:
try:
    unique_counts_df = analyze_uniqueness(df[new_features])
except ValueError as e:
    print(f"Error: {e}")

Here we can see columns with 100% and 0% variability i.e. every row is unique or contains the same number. This is typical of unique identifiers and in our case policy codes. We can remove these as they won't be of any use to us for further analysis.

To simplify our data we'll remove 100%, 0%, and anythin less than 5%:

- 100% of values are unique = `id`, `url`
- 0% of values are unique = `policy_code`, `pymnt_plan`

We'll also take this opportunity to remove `unnamed:_0.1`, `unnamed:_0` too.

In [ ]:
# List of columns to drop
columns_to_drop = ['policy_code', 'pymnt_plan']

# Remove specified columns from pre_hardship_fields_clean
new_features = [col for col in new_features if col not in columns_to_drop]

In [ ]:
print(df.shape)
print(len(new_features))

Our feature list has been updated with the new columns. This means that for each column with missing values, a new binary indicator column is created, where `1` represents a missing value and `0` indicates that a value was present. 

This approach ensures that we retain as much information as possible from the original dataset while also capturing the fact that missing values themselves may provide valuable insight. For instance, missing income information could be an indicator of a higher default risk.

## Target Feature Engineering

Let's start by defining our target variable and applying simplify the `loan_status`

In [ ]:
df['loan_status'].value_counts()

In [ ]:
# Define the logical groupings for 'loan_status'
loan_status_groupings = {
    'Fully Paid': 'Paid Loan',
    'Current': 'Active Loan',
    'Charged Off': 'Defaulted Loan',
    'Late (31-120 days)': 'Late Loan',
    'In Grace Period': 'Late Loan',
    'Late (16-30 days)': 'Late Loan',
    'Does not meet the credit policy. Status:Fully Paid': 'Paid Loan',
    'Issued': 'Active Loan',
    'Does not meet the credit policy. Status:Charged Off': 'Defaulted Loan',
    'Default': 'Defaulted Loan'
}

# Apply the grouping to the 'loan_status' column
df['loan_status_grouped_kn'] = df['loan_status'].replace(loan_status_groupings)

# Verify the groupings
print(df['loan_status_grouped_kn'].value_counts())

In [ ]:
new_features.extend(['loan_status_grouped_kn'])

In [ ]:
print(len(new_features))

## Our dependant variable

Our dependant veriable is `loan_status_grouped_kn` further work is to understand how best to predict good and bad loan applicants.

In [ ]:
df['loan_status_grouped_kn'].head()

In [ ]:
# Check if the specified columns are in new_features
columns_to_check = ['loan_status_grouped_kn']
missing_columns = [col for col in columns_to_check if col not in new_features]

# Display results
if not missing_columns:
    print("Both columns are in new_features.")
else:
    print(f"The following columns are missing from new_features: {missing_columns}")

In [ ]:
df_dropped = df[new_features].copy()

Let's check our data frame to see what we're working with. Note our most recent data frame is now df_dropped and no longer df.

In [ ]:
print(df_dropped.shape)
print(len(new_features))

### Exploratory Data Analysis (EDA) on Missing Values

To understand the impact of missing values on our target variable (`loan_status`), we perform an exploratory analysis. 

This compares the distribution of loan status between rows where key variables are missing and where they are not. By doing this, we hope to detect which missing data is associated with loan outcomes. 

This helps us understand how big a influence on the target variable.


In [ ]:
# Select columns that have missing values in df_dropped
missing_cols = df_dropped.columns[df_dropped.isnull().any()]

# Store results for plotting
missing_dict = {}
not_missing_dict = {}

# Function to collect percentages for missing and non-missing data
def missing_value_analysis(column, target_column='loan_status_grouped_kn'):
    missing = df_dropped[df_dropped[column].isnull()][target_column].value_counts(normalize=True) * 100
    not_missing = df_dropped[df_dropped[column].notnull()][target_column].value_counts(normalize=True) * 100
    missing_dict[column] = missing
    not_missing_dict[column] = not_missing

# Apply the function for all columns with missing data
for col in missing_cols:
    missing_value_analysis(col)

# Create DataFrames for heatmaps
missing_df = pd.DataFrame(missing_dict).fillna(0)  # Fill NaN with 0 to ensure proper heatmap display
not_missing_df = pd.DataFrame(not_missing_dict).fillna(0)

# Plotting heatmaps one below the other
fig, ax = plt.subplots(2, 1, figsize=(12, 16), gridspec_kw={'height_ratios': [1, 1]})  # Adjust aspect ratio

# Heatmap for missing data
sns.heatmap(missing_df, annot=False, cmap="Blues", ax=ax[0], cbar_kws={"shrink": .75})
ax[0].set_title('Percentage of Loan Status for Missing Data')
ax[0].tick_params(axis='x', rotation=90, labelsize=10)  # Rotate x-axis labels for readability
ax[0].tick_params(axis='y', labelsize=10)  # Adjust y-axis label size

# Heatmap for non-missing data
sns.heatmap(not_missing_df, annot=False, cmap="Greens", ax=ax[1], cbar_kws={"shrink": .75})
ax[1].set_title('Percentage of Loan Status for Non-Missing Data')
ax[1].tick_params(axis='x', rotation=90, labelsize=10)  # Rotate x-axis labels for readability
ax[1].tick_params(axis='y', labelsize=10)  # Adjust y-axis label size

# Adjust layout to prevent overlap
plt.tight_layout()
plt.show()

### Summary of Missing and Non-Missing Data by Loan Status

#### 1. Percentage of Missing Data (First Image)
- **Paid Loan**: Shows a high percentage of missing data across numerous columns, particularly towards the end of the list (indicated by darker shades of blue).
- **Active Loan** and **Late Loan**: Generally have less missing data, though some specific columns still contain notable missing values.
- **Defaulted Loan**: Contains fewer missing values compared to `Paid Loan`, but some columns still show missing data.

#### 2. Percentage of Non-Missing Data (Second Image)
- **Paid Loan**: Displays lower percentages of non-missing data across various columns, aligning with the missing data heatmap (darker greens represent more missing data).
- **Active Loan** and **Late Loan**: Feature higher percentages of non-missing data in most columns, shown by lighter green shades, suggesting more complete data.
- **Defaulted Loan**: Generally has a higher percentage of non-missing data than `Paid Loan`, but less than `Active Loan` and `Late Loan`.

### Data Preparation and Cleaning

Perform thorough data cleaning on the provided dataset, including but not limited to the following steps:

* Handling missing values (imputation or removal)
* Converting data types to appropriate formats
* Removing duplicate records
* Detecting and handling outliers
* Encoding categorical variables

In [ ]:
# Cross table of data types in df
dtype_crosstab = df_dropped.dtypes.value_counts().reset_index()
dtype_crosstab.columns = ['Data Type', 'Count']

# Display the cross table
dtype_crosstab

In [ ]:
# Correct references to 'completed' to 'complete' in the 'hardship_status' column
df_dropped['hardship_status_kn'] = df['hardship_status'].replace('COMPLETED', 'COMPLETE')

In [ ]:
new_features.extend(['hardship_status_kn'])

In [ ]:
print(df_dropped.shape)
print(len(new_features))

In [ ]:
new_features

### Preliminary Feature Selection

Based on our initial quick analysis of the data and the data dictionary, we'll now work on the shortlisted features for further exploration in our machine learning model. These features were selected based on their relevant at the time of taking out a loan.

In [ ]:
df_dropped.shape

In [ ]:
# Filter and list columns with 'object' data types from our selected features
# This helps us identify which columns require encoding or conversions before modeling
df_dropped[new_features].select_dtypes(include=['object']).dtypes

In [ ]:
df_dropped[new_features].select_dtypes(include=['object']).head()

## List split

Let's take our list of features we'd like to review for machine learning and split them into 3 seperate lists (boolean, numerical, and categorical) for further analysis.

In this next phase I want to explore the feature selection for multicollinearity and distribution.

In [ ]:
def split_data_frame(features_list, df):
    """
    Splits the provided DataFrame into three lists containing Boolean, Numerical, and Categorical column names.
    Converts floats with trailing zeros into integers and replaces NaN values with 0 for integers, 0.00 for floats.

    Parameters:
    features_list (list): List of column names to be checked.
    df (pd.DataFrame): The input DataFrame to split.

    Returns:
    tuple: A tuple containing three lists (boolean_cols, numerical_cols, categorical_cols).
    """
    boolean_cols = []
    numerical_cols = []
    categorical_cols = []

    # Define acceptable boolean values
    acceptable_boolean_values = {0, 1, True, False, 0.0, 1.0}

    for col in features_list:
        # Treat each column explicitly as a Series
        column_series = df[col]

        # Handle cases where columns might be interpreted incorrectly
        if pd.api.types.is_bool_dtype(column_series) or all(column_series.dropna().isin(acceptable_boolean_values)):
            boolean_cols.append(col)
        elif pd.api.types.is_numeric_dtype(column_series):
            # Check for floats with trailing zeros
            if column_series.dtype == 'float64':
                # Check if all float values are equivalent to integers
                if all(column_series.dropna() == column_series.dropna().astype(int)):
                    df[col] = column_series.fillna(0).astype(int)  # Replace NaNs with 0 and convert to int
                else:
                    df[col] = column_series.fillna(0.00)  # Replace NaNs with 0.00 for floats
                numerical_cols.append(col)
            else:
                df[col] = column_series.fillna(0)  # Replace NaNs with 0 for integers
                numerical_cols.append(col)
        else:
            categorical_cols.append(col)
    
    # Print a summary of the count of columns in each list
    print(f"Summary of column counts:")
    print(f"boolean_list contains {len(boolean_cols)} values")
    print(f"numerical_list contains {len(numerical_cols)} values")
    print(f"categorical_list contains {len(categorical_cols)} values")
    print(f"The feature list we'll be working with contains {df_dropped[new_features].shape} rows and columns.")

    return boolean_cols, numerical_cols, categorical_cols

# Instructions:
# this calls the split_data_frame function above create three lists to capture the sorting outputs in. 
# These will later be used to pull some graphs to evaluate the data and what possible transformations we've missed.
# boolean_list, numerical_list, categorical_list = split_data_frame(new_features, df_dropped)


In [ ]:
# Check which columns are in df_dropped
missing_columns = [col for col in new_features if col not in df_dropped.columns]

print("Missing columns:", missing_columns)

In [ ]:
# this calls the split_data_frame function above create three lists to capture the sorting outputs in. 
# These will later be used to pull some graphs to evaluate the data and what possible transformations we've missed.
# Our function parameters are list and the latest version of our data frame in this caes new_features and df_dropped accordingly.
boolean_list, numerical_list, categorical_list = split_data_frame(new_features, df_dropped)

In [ ]:
df_dropped[categorical_list].head()

In [ ]:
df_dropped[numerical_list].head()

In [ ]:
df_dropped[boolean_list].head()

The following sections are the code snippets we'll use to analyse our data (Numerical, Boolean, Categorical)

##### Numerical Processing

In [ ]:
def analyze_numeric_columns(numeric_cols, dataframe):
    """
    Analyze and visualize numeric columns in a DataFrame.

    Parameters:
    numeric_cols (list): List of numeric column names to analyze.
    dataframe (pd.DataFrame): The DataFrame containing the data.
    """
    for column in numeric_cols:
        print(f"\nSummary Statistics and Analysis for Numeric Column: {column}")

        # Check if the column exists in the DataFrame
        if column not in dataframe.columns:
            print(f" '{column}' is not found in the DataFrame. Skipping...")
            continue

        # Ensure the column is converted to numeric
        dataframe.loc[:, column] = pd.to_numeric(dataframe[column], errors='coerce')

        # Drop NaN values to ensure we have numeric data for analysis
        numeric_data = dataframe[column].dropna()

        if numeric_data.empty:
            print(f"No valid numeric data available for column: {column}. Skipping...")
            continue  # Skip the column if there's no valid data

        # Calculate z-scores
        z_scores = (numeric_data - numeric_data.mean()) / numeric_data.std()

        # Calculate IQR
        Q1 = numeric_data.quantile(0.25)
        Q3 = numeric_data.quantile(0.75)
        IQR = Q3 - Q1

        # Identify outliers based on z-scores (z > 3 or z < -3)
        outliers_z = numeric_data[(z_scores > 3) | (z_scores < -3)]

        # Identify outliers based on IQR
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outliers_iqr = numeric_data[(numeric_data < lower_bound) | (numeric_data > upper_bound)]

        # Create a figure for the distribution and box plot
        fig, axs = plt.subplots(1, 2, figsize=(FIG_WIDTH, FIG_HEIGHT))

        # Distribution plot (histogram)
        sns.histplot(numeric_data, kde=True, bins=10, ax=axs[0])
        axs[0].set_title(f'Distribution of {column}')
        axs[0].set_xlabel(column)
        axs[0].set_ylabel('Frequency')

        # Box plot for outlier detection
        sns.boxplot(x=numeric_data, ax=axs[1])
        axs[1].set_title(f'Box Plot for {column} (Outlier Detection)')
        axs[1].set_xlabel(column)

        # Adjust layout to prevent overlap
        plt.tight_layout()
        plt.show()

        # Summary Statistics
        print(f"\nSummary Statistics for Numeric  '{column}':")
        print(numeric_data.describe())

        # Interquartile Range (IQR)
        print(f"\nInterquartile Range (IQR): {IQR:.4f}")
        print(f"Lower Bound for Outliers (IQR method): {lower_bound:.4f}")
        print(f"Upper Bound for Outliers (IQR method): {upper_bound:.4f}")
        print(f"Number of Outliers (IQR method): {len(outliers_iqr)}")

        # Display outliers based on IQR
        #if not outliers_iqr.empty:
        #    print("\nOutliers detected using the IQR method:")
        #    print(outliers_iqr)
        #else:
        #    print("\nNo outliers detected using the IQR method.")

        # Z-scores
        print("\nZ-score Summary:")
        print(z_scores.describe())

        print(f"\nNumber of Outliers (Z-score method): {len(outliers_z)}")

        # Display outliers based on Z-scores
        #if not outliers_z.empty:
        #    print("\nOutliers detected using the Z-score method:")
        #    print(outliers_z)
        #else:
        #    print("\nNo outliers detected using the Z-score method.")

        # Skewness
        skewness_value = skew(numeric_data)
        print(f"\nSkewness: {skewness_value:.4f}")

        # Normality Tests
        print("\nNormality Tests:")
        # D'Agostino's K^2 Test
        k2_stat, k2_p = normaltest(numeric_data)
        print(f"D'Agostino's K^2 Test: Statistic={k2_stat:.4f}, p-value={k2_p:.4f}")

        # Shapiro-Wilk Test
        shapiro_stat, shapiro_p = shapiro(numeric_data)
        print(f"Shapiro-Wilk Test: Statistic={shapiro_stat:.4f}, p-value={shapiro_p:.4f}")

        # Anderson-Darling Test
        anderson_result = anderson(numeric_data)
        print(f"Anderson-Darling Test: Statistic={anderson_result.statistic:.4f}")
        #for i in range(len(anderson_result.critical_values)):
        #    sl, cv = anderson_result.significance_level[i], anderson_result.critical_values[i]
        #    if anderson_result.statistic < cv:
        #        result = "Accept"
        #    else:
        #        result = "Reject"
        #    print(f"At {sl}% significance level, critical value: {cv:.4f}, {result} the null hypothesis of normality")

        # Kolmogorov-Smirnov Test against normal distribution
        # ks_stat, ks_p = kstest(numeric_data, 'norm', args=(numeric_data.mean(), numeric_data.std()))
        # print(f"Kolmogorov-Smirnov Test: Statistic={ks_stat:.4f}, p-value={ks_p:.4f}")


##### Boolean Processing

In [ ]:
def analyze_boolean_columns(boolean_cols, dataframe):
    """
    Analyze and visualize boolean columns in a DataFrame.

    Parameters:
    boolean_cols (list): List of boolean column names to analyze.
    dataframe (pd.DataFrame): The DataFrame containing the data.
    """
    for column in boolean_cols:
        print(f"\nSummary Statistics and Analysis for Boolean Column: {column}")

        # Check if the column exists in the DataFrame
        if column not in dataframe.columns:
            print(f"'{column}' is not found in the DataFrame. Skipping...")
            continue

        # Cast the column to boolean in case it contains 1/0 or other non-boolean values
        dataframe[column] = dataframe[column].astype(bool)

        # Prepare boolean counts
        boolean_counts = dataframe[column].value_counts()

        # Create a figure with two subplots
        fig, axs = plt.subplots(1, 2, figsize=(FIG_WIDTH, FIG_HEIGHT)) # Configured at the beginning of the file for image consistancy

        # Bar plot using 'x' parameter
        sns.countplot(
            x=column,
            data=dataframe,
            ax=axs[0]
        )
        axs[0].set_title(f'Boolean Distribution for {column}')
        axs[0].set_xlabel(column)
        axs[0].set_ylabel('Count')

        # Rotate x-axis labels
        plt.xticks(rotation=90)

        # Pie chart on the right subplot
        boolean_counts.plot.pie(
            autopct='%1.1f%%',
            ax=axs[1],
            startangle=90
        )
        axs[1].set_title(f'Proportion of True/False for {column}')
        axs[1].set_ylabel('')

        # Adjust layout to prevent overlap
        plt.tight_layout()
        plt.show()

        # Display summary statistics for boolean data
        print(f"\nSummary for Boolean '{column}':")
        true_count = boolean_counts.get(True, 0)
        false_count = boolean_counts.get(False, 0)
        total_count = true_count + false_count
        true_percentage = (true_count / total_count) * 100 if total_count > 0 else 0

        print(f"Count of True: {true_count}")
        print(f"Count of False: {false_count}")
        print(f"Percentage of True: {true_percentage:.2f}%")


##### Categorical Processing

In [ ]:
def analyze_categorical_columns(categorical_cols, dataframe):
    """
    Analyze and visualize categorical columns in a DataFrame.

    Parameters:
    categorical_cols (list): List of categorical column names to analyze.
    dataframe (pd.DataFrame): The DataFrame containing the data.
    """
    for column in categorical_cols:
        print(f"\nSummary Statistics and Analysis for Categorical Column: {column}")

        # Check if the column exists in the DataFrame
        if column not in dataframe.columns:
            print(f" '{column}' is not found in the DataFrame. Skipping...")
            continue

        # Prepare category counts and percentages
        category_counts = dataframe[column].value_counts()
        category_percentages = dataframe[column].value_counts(normalize=True) * 100

        # Display bar plot and pie chart
        fig, axs = plt.subplots(1, 2, figsize=(FIG_WIDTH, FIG_HEIGHT))

        # Bar plot using 'x' parameter
        sns.countplot(
            x=column,
            data=dataframe,
            ax=axs[0]
        )
        axs[0].set_title(f'Frequency Distribution for {column} (Categorical Data)')
        axs[0].set_xlabel(column)
        axs[0].set_ylabel('Frequency')
        plt.xticks(rotation=45, ha='right')

        # Pie chart on the right subplot
        category_counts.plot.pie(
            autopct='%1.1f%%',
            ax=axs[1],
            title=f'Proportion of Categories for {column}',
            startangle=90
        )
        axs[1].set_ylabel('')

        # Adjust layout to prevent overlap
        plt.tight_layout()
        plt.show()

        print(f"\nFrequency Table for '{column}':")
        freq_table = pd.DataFrame({'Count': category_counts, 'Percentage': category_percentages})
        print(freq_table)


In [ ]:
len(new_features) # issue

Let's start with our analysis of the Categorical features.

In [ ]:
import re

# List to store features that contain any numerical values (even within mixed text)
categorical_list_with_numerical_values = []

# Iterate through each column in categorical_list
for column in categorical_list:
    # Check if the column contains any values with numerical characters
    if df_dropped[column].apply(lambda x: bool(re.search(r'\d', str(x)))).any():
        categorical_list_with_numerical_values.append(column)
        print(f" '{column}' contains numerical values.")

In [ ]:
df_dropped[categorical_list_with_numerical_values].columns

Let's explore our data to avoid creating graphs for rows that contain unique values. To do this we'll run a bar chart.

In [ ]:
def analyze_categorical_uniqueness(df, columns):
    """
    Analyzes the number of unique values in categorical columns and 
    helps identify suitable candidates for visualization.

    Args:
        df: The Pandas DataFrame.
        columns: A list of column names to analyze.

    Returns:
        A dictionary where keys are column names and values are 
        the number of unique values in each column. Also, 
        prints a summary to assist visualizing the results via bar graphs.
    """
    uniqueness_counts = {}
    for col in columns:
        if not pd.api.types.is_object_dtype(df[col]):  # checks for object type; adjust if needed
            continue  # Skip analysis for non-categorical columns
        
        unique_count = df[col].nunique()  # use nunique for direct count of unique values
        uniqueness_counts[col] = unique_count

    # Sort the dictionary by unique counts in descending order
    sorted_uniqueness_counts = dict(sorted(uniqueness_counts.items(), key=lambda item: item[1], reverse=True))

    # Print sorted output with unique counts
    print("Sorted Unique Value Counts:")
    for col, unique_count in sorted_uniqueness_counts.items():
        print(f"Number of unique values in '{col}': {unique_count}")

    # Create a bar graph to visualize the number of unique values, sorted from high to low
    plt.figure(figsize=(12, 6))
    plt.bar(sorted_uniqueness_counts.keys(), sorted_uniqueness_counts.values())
    plt.xlabel("Categorical Columns", fontsize=14)
    plt.ylabel("Number of Unique Values", fontsize=14)
    plt.title("Unique Value Counts per Categorical Column (Sorted)", fontsize=16)
    plt.xticks(rotation=45, ha="right", fontsize=10)  # Rotate x-axis labels for readability
    plt.tight_layout()  # Adjust subplot parameters for a tight layout
    plt.show()

    return sorted_uniqueness_counts

# Example usage:

#categorical_uniqueness = analyze_categorical_uniqueness(df_dropped, categorical_list_with_numerical_values)
#columns_for_visualization = [col for col, count in categorical_uniqueness.items() if count <= 15] # Example: Use only below or equal to 10
#print(f"\nSuggested columns for visualization (<= 15 unique values): {columns_for_visualization}")


In [ ]:
print(df_dropped.shape)
print(len(new_features))

In [ ]:
categorical_uniqueness = analyze_categorical_uniqueness(df, categorical_list_with_numerical_values)

columns_for_visualization = [col for col, count in categorical_uniqueness.items() if count <= 10] # Example: Use only below or equal to 10

Straight away we can see `int_rate` (584),`emp_title` (40049), `issue_d` (159),`title` (3454), `revol_util` (1088), `last_pymnt_d` (147), `earliest_cr_line` (667), `sec_app_earliest_cr_line` (506), `last_credit_pull` (137). `zip_code` (878) stand out, we'll remove these as graphically representing these won't produce any meaningful insight. However, we'll see if we can reduce them into logical groupings later on.

In [ ]:
exclude_list = ['int_rate', 'zip_code', 'emp_title', 'issue_d', 'title', 'revol_util', 'last_pymnt_d', 'earliest_cr_line', 'sec_app_earliest_cr_line', 'last_credit_pull_d']

# Filter the categorical columns to exclude specified ones
filtered_categorical_columns = [col for col in categorical_list_with_numerical_values if col not in exclude_list]

# Analyze the filtered categorical columns
analyze_categorical_columns(filtered_categorical_columns, df_dropped)

### Handling Object and Float Features

We have identified the following columns that require conversion or encoding:

#### Actions Post-Analysis

##### Convert String to Integer
- **term**: Extract numerical part and convert to integer (keep `36` and `60`).
- **emp_length**: Extract numerical years and convert to integer.

##### Convert String to Float
- **int_rate**: Convert to float after removing any non-numeric characters.
- **revol_util**: Convert to float after removing the "%" symbol.

##### Encode Categorical Values
- **sub_grade**: Use as is or encode if necessary; consider dropping **grade** if redundant.
- **loan_status**: Group or encode based on loan status levels.
- **hardship_loan_status**: Analyze and group similar hardship statuses if logical.

##### Convert to Date/Time Format
- **issue_d**: Convert to date/time for chronological analysis.
- **earliest_cr_line**: Convert to date/time to track the earliest credit history.
- **last_pymnt_d**: Convert to date/time; create separate year and month columns.
- **next_pymnt_d**: Convert to date/time; add year and month columns.
- **last_credit_pull_d**: Convert to date/time for recent credit activity insights.
- **sec_app_earliest_cr_line**: Convert to date/time for secondary applicants’ credit history.
- **hardship_start_date**: Convert to date/time; add year and month columns.
- **hardship_end_date**: Convert to date/time; add year and month columns.
- **payment_plan_start_date**: Convert to date/time; add year and month columns.

##### Remove Non-Analytical or Irrelevant Columns
- **emp_title**: Not relevant for numerical analysis; remove.
- **url**: Non-analytical; remove as it doesn’t contribute to analysis.

##### Evaluate for Categorical Consistency
- **zip_code**: Analyze the first few digits if relevant to extract location-based insights.


In [ ]:
print(df_dropped.shape)
print(len(new_features))

In [ ]:
def update_feature_list(original_features, new_features):
    """
    Checks for the existence of original features in a list, 
    removes them, and adds new features.

    Parameters:
    -----------
    original_features : list
        List of original feature names to be removed.
    new_features : list
        List of new feature names to be added.

    Returns:
    --------
    list
        Updated list of features.
    """

    updated_features = original_features.copy()  # Create a copy to avoid modifying the original list

    for feature in original_features:
        if feature in updated_features:
            updated_features.remove(feature)

    updated_features.extend(new_features)
    return updated_features

# Example usage:
#original_features = ['int_rate', 'revol_util'] 
#new_features = ['int_rate_kn', 'revol_util_kn']

#updated_list = update_feature_list(original_features, new_features)
#print(updated_list)  # Output: ['int_rate_kn', 'revol_util_kn']

#### Convert string to integer

In [ ]:
import re

# Columns that need to be converted from string to integer
string_columns_to_convert_int = ['term', 'emp_length']  # features to replace with int values

# Convert each specified column to an integer in a new column
for column in string_columns_to_convert_int:
    # Extract numerical part and convert to integer
    df_dropped[f"{column}_kn"] = df_dropped[column].apply(lambda x: int(re.search(r'\d+', str(x)).group()) if pd.notnull(x) else None)

In [ ]:
df_dropped[['term_kn','emp_length_kn']].head()

In [ ]:
# List of columns to remove
columns_to_remove = ['term', 'emp_length']

# Remove specified columns from new_features if they exist
new_features = [col for col in new_features if col not in columns_to_remove]

In [ ]:
new_features.extend(['term_kn', 'emp_length_kn'])

In [ ]:
# Check if the specified columns are in new_features
columns_to_check = string_columns_to_convert_int
missing_columns = [col for col in columns_to_check if col not in new_features]

# Display results
if not missing_columns:
    print("Both columns are in new_features.")
else:
    print(f"The following columns are missing from new_features: {missing_columns}")

In [ ]:
print(df_dropped.shape)
print(len(new_features))

#### Convert string to float

In [ ]:
import re

# Define the columns that need to be converted from string to float
string_columns_to_convert_float = ['int_rate', 'revol_util']  # features to replace with float values

# Convert each specified column to a float in a new column as percentage (e.g., 80% becomes 0.80)
for column in string_columns_to_convert_float:
    # Extract numerical part, convert to float, and divide by 100
    df_dropped[f"{column}_kn"] = df_dropped[column].apply(
        lambda x: float(re.search(r'\d+', str(x)).group()) / 100 if pd.notnull(x) else None
    )

In [ ]:
df_dropped['int_rate_kn'].head()

In [ ]:
new_features.extend(['int_rate_kn', 'revol_util_kn'])

In [ ]:
# List of columns to remove
columns_to_remove = ['int_rate', 'revol_util']

# Remove specified columns from new_features if they exist
new_features = [col for col in new_features if col not in columns_to_remove]

In [ ]:
# Check if the specified columns are in new_features
columns_to_check = ['int_rate_kn', 'revol_util_kn']
missing_columns = [col for col in columns_to_check if col not in new_features]

# Display results
if not missing_columns:
    print("Both columns are in new_features.")
else:
    print(f"The following columns are missing from new_features: {missing_columns}")

In [ ]:
print(df_dropped.shape)
print(len(new_features))

#### Convert date time

In [ ]:
df_dropped[['issue_d', 'earliest_cr_line']].head()

In [ ]:
# List of date columns to convert
date_columns = ['issue_d', 'earliest_cr_line']

# Convert the columns to datetime format
for col in date_columns:
    # Convert to datetime using the format '%b-%y', with errors coerced to NaT
    df_dropped[col] = pd.to_datetime(df_dropped[col], format='%b-%y', errors='coerce')

# Extracting date features for each date column
for col in date_columns:
    # Define new column names for year and month
    year_col = f'{col}_year_kn'
    month_col = f'{col}_month_kn'
       
    # Extract year and month, convert to integer with support for NaNs
    df_dropped[year_col] = df_dropped[col].dt.year.astype('Int64').fillna(0)  # Extract year
    df_dropped[month_col] = df_dropped[col].dt.month.astype('Int64').fillna(0)  # Extract month
    
    # Add the new column names to the existing 'new_features' list
    new_features.extend([year_col, month_col])
    
    # Print the names of the newly created columns
    print(f"New columns created: {year_col}, {month_col}")


In [ ]:
df_dropped[['issue_d', 'earliest_cr_line', 'issue_d_year_kn', 'issue_d_month_kn', 'earliest_cr_line_year_kn', 'earliest_cr_line_month_kn']].head()

In [ ]:
new_features.extend(['issue_d_year_kn', 'issue_d_month_kn','earliest_cr_line_year_kn', 'earliest_cr_line_month_kn'])

In [ ]:
# List of columns to remove
columns_to_remove = ['issue_d', 'earliest_cr_line']

# Remove specified columns from new_features if they exist
new_features = [col for col in new_features if col not in columns_to_remove]

In [ ]:
# Check if the specified columns are in new_features
columns_to_check = ['issue_d_year_kn', 'issue_d_month_kn','earliest_cr_line_year_kn', 'earliest_cr_line_month_kn']
missing_columns = [col for col in columns_to_check if col not in new_features]

# Display results
if not missing_columns:
    print("All columns are in new_features.")
else:
    print(f"The following columns are missing from new_features: {missing_columns}")

In [ ]:
print(df_dropped.shape)
print(len(new_features))

In [ ]:
new_features

In [ ]:
# Get the set of columns in df_dropped
df_dropped_columns = set(df_dropped.columns)

# Convert new_features to a set for comparison
new_features_set = set(new_features)

# Find columns in new_features that are missing in df_dropped
missing_in_df_dropped = new_features_set - df_dropped_columns

# Find columns in df_dropped that are not in new_features (if relevant)
extra_in_df_dropped = df_dropped_columns - new_features_set

# Print the results
print("Features in new_features missing from df_dropped:", missing_in_df_dropped)
print("Columns in df_dropped not listed in new_features:", extra_in_df_dropped)

#### Remove Non Analytical or Irrelevent columns

I'm tempted to leave these in as it could be a good indicator of good or bad payment trends but won't have the time to do it for now we'll remove them.

In [ ]:
# List of columns to remove
columns_to_remove = ['zip_code', 'emp_title']

# Remove specified columns from new_features if they exist
new_features = [col for col in new_features if col not in columns_to_remove]

In [ ]:
# sense check
print(df_dropped.shape)
print(len(new_features))

#### Creating logical groups 

- **hardship_reason**: Analyse and group to simplify analysis (import from df)

In [ ]:
# Define the logical groupings for 'hardship_reason'
hardship_reason_groupings = {
    'INCOMECURT': 'Income Loss',
    'UNEMPLOYED': 'Income Loss',
    'UNEMPLOYMENT': 'Income Loss',
    'INCOME_CURTAILMENT': 'Income Loss',
    'REDCDHOURS': 'Income Loss',
    'REDUCED_HOURS': 'Income Loss',
    'FURLOUGH': 'Income Loss',
    'MEDICAL': 'Health Issues',
    'DISABILITY': 'Health Issues',
    'NATURAL_DISASTER': 'External Events',
    'NATDISAST': 'External Events',
    'FINANCIAL': 'Financial Strain',
    'EXCESSIVE_OBLIGATIONS': 'Financial Strain',
    'EXCESSOBLI': 'Financial Strain',
    'DIVORCE': 'Family Circumstances',
    'FAMILY_DEATH': 'Family Circumstances',
    'DEATH': 'Family Circumstances'
}

# Apply the grouping to the 'hardship_reason' column
df_dropped['hardship_reason_grouped_kn'] = df['hardship_reason'].replace(hardship_reason_groupings) # adding this back into our data set

# Verify the groupings
print(df_dropped['hardship_reason_grouped_kn'].value_counts())


In [ ]:
# Define the logical groupings for 'loan_status'
loan_status_groupings = {
    'Fully Paid': 'Completed',
    'Current': 'In Progress',
    'Charged Off': 'Defaulted',
    'Late (31-120 days)': 'Late',
    'In Grace Period': 'Late',
    'Late (16-30 days)': 'Late',
    'Does not meet the credit policy. Status:Fully Paid': 'Completed',
    'Issued': 'In Progress',
    'Does not meet the credit policy. Status:Charged Off': 'Defaulted',
    'Default': 'Defaulted'
}

# Apply the grouping to the 'loan_status' column
df_dropped['loan_status_grouped_kn'] = df['loan_status'].replace(loan_status_groupings)

# Verify the groupings
print(df_dropped['loan_status_grouped_kn'].value_counts())

In [ ]:
# Replace 'Any', 'Other', and 'none' with a unified value 'Other'
df_dropped['home_ownership_grouped_kn'] = df_dropped['home_ownership'].replace(['ANY', 'OTHER', 'NONE'], 'OTHER')

# Display the updated DataFrame to verify
df_dropped['home_ownership_grouped_kn'].value_counts()

In [ ]:
new_features.extend(['hardship_reason_grouped_kn', 'loan_status_grouped_kn','home_ownership_grouped_kn'])

In [ ]:
# Define a list of columns to be removed
# these are all the columns we've altered. These can remain in the data frame but should be removed from our feature list
columns_to_remove = ['hardship_reason', 'loan_status','home_ownership']

# Remove specified columns from new_features if they exist
new_features = [col for col in new_features if col not in columns_to_remove]

In [ ]:
# Check if the specified columns are in new_features
columns_to_check = ['hardship_reason_grouped_kn', 'loan_status_grouped_kn','home_ownership_grouped_kn']
missing_columns = [col for col in columns_to_check if col not in new_features]

# Display results
if not missing_columns:
    print("Both columns are in new_features.")
else:
    print(f"The following columns are missing from new_features: {missing_columns}")

In [ ]:
print(df_dropped.shape)
print(len(new_features))

## EDA

Let's run our list function again on the udpated data frame and scroll through to update our feature list.

In [ ]:
analyze_boolean_columns(boolean_list, df_dropped)

In [ ]:
# List of columns to check for missing values
columns_to_check = [
    'earliest_cr_line', 'fico_range_high', 
    'fico_range_low', 'last_fico_range_high', 
    'last_fico_range_low', 'pub_rec', 
    'pub_rec_bankruptcies', 'annual_inc', 
    'inq_last_6mths'
]

# Check for missing values in each specified column and display the count
for column in columns_to_check:
    missing_count = df[column].isnull().sum() # checking against original data if 0 we can remove these from our ML feature list
    print(f"Missing values in '{column}': {missing_count}")

After reviewing our output I've decided to drop the following values as their representation within the data set is not meaningful enought to be any statistical significance.

In [ ]:
# Original list of columns to remove
columns_to_remove = [
    'earliest_cr_line_missing_clean_kn', 'fico_range_high_missing_clean_kn', 
    'fico_range_low_missing_clean_kn', 'last_fico_range_high_missing_clean_kn', 
    'last_fico_range_low_missing_clean_kn', 'pub_rec_missing_clean_kn', 
    'annual_inc_missing_clean_kn', 'inq_last_6mths_missing_clean_kn'
]

# Create a new list by excluding the columns in columns_to_remove
new_features_updated = [col for col in new_features if col not in columns_to_remove]

In [ ]:
df_dropped['loan_status_grouped_kn'].value_counts()

Let's take a look at our numerical data

In [ ]:
analyze_numeric_columns(numerical_list, df_dropped)

Now that we've finised our EDA let's check some of the categorical points with our target variable

In [ ]:
# Create a cross-tabulation of purpose and loan_status
comparison_loan_status_purpose = pd.crosstab(df_dropped['loan_status_grouped_kn'], df_dropped['purpose'])

# Display the result
comparison_loan_status_purpose

In [ ]:
import matplotlib.pyplot as plt

# Sort data by total loan counts across all statuses for each purpose (descending order)
sorted_data = comparison_loan_status_purpose.sum(axis=0).sort_values(ascending=False)
sorted_columns = sorted_data.index
comparison_loan_status_purpose_sorted = comparison_loan_status_purpose[sorted_columns]

# Plot a stacked bar chart
fig, ax = plt.subplots(figsize=(12, 8))

# Plot each loan status as a stacked segment
comparison_loan_status_purpose_sorted.T.plot(kind='bar', stacked=True, ax=ax, colormap='viridis')

# Set labels and title
ax.set_xlabel('Purpose')
ax.set_ylabel('Number of Loans')
ax.set_title('Distribution of Loan Statuses by Loan Purpose')

# Rotate x-axis labels for readability
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.legend(title='Loan Status')

# Display the plot
plt.show()


In [ ]:
# Create a cross-tabulation of purpose and loan_status
comparison_loan_status_ver_status = pd.crosstab(df_dropped['loan_status_grouped_kn'], df_dropped['verification_status'])

# Display the result
comparison_loan_status_ver_status

In [ ]:
import matplotlib.pyplot as plt

# Plot a stacked bar chart
fig, ax = plt.subplots(figsize=(10, 6))

# Transpose the DataFrame for easier plotting and plot each loan status as a stacked segment
comparison_loan_status_ver_status.T.plot(kind='bar', stacked=True, ax=ax, colormap='viridis')

# Set labels and title
ax.set_xlabel('Verification Status')
ax.set_ylabel('Number of Loans')
ax.set_title('Distribution of Loan Statuses by Verification Status')

# Rotate x-axis labels for readability
plt.xticks(rotation=0)
plt.tight_layout()
plt.legend(title='Loan Status')

# Display the plot
plt.show()

This is an interesting view, as the the Not Verified, and Source Verified represent both roughly 15-20% of the charged off loans for each status but Verified accounts for roughly 23% of the total. I was expecting a lot of the Charged Off in the Not Verified `verification_status`.

In [ ]:
# Create a cross-tabulation of purpose and loan_status
comparison_loan_status_addr_state = pd.crosstab(df_dropped['loan_status_grouped_kn'], df_dropped['addr_state'])

# Display the result
comparison_loan_status_addr_state

In [ ]:
# Filter data to only include the 'Defaulted' loan status
defaulted_by_state = comparison_loan_status_addr_state.loc['Defaulted']

# Sort the data by the number of defaulted loans in descending order
defaulted_by_state_sorted = defaulted_by_state.sort_values(ascending=False)

# Plot the data
plt.figure(figsize=(12, 8))
defaulted_by_state_sorted.plot(kind='bar', color='salmon', edgecolor='black')

# Set labels and title
plt.xlabel("State")
plt.ylabel("Number of Defaulted Loans")
plt.title("Number of Defaulted Loans by State (Sorted)")

# Rotate x-axis labels for readability
plt.xticks(rotation=45, ha='right')
plt.tight_layout()

# Show the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Group data by 'loan_status' and 'addr_state' and count occurrences
grouped_data = df_dropped.groupby(['addr_state', 'loan_status_grouped_kn']).size().unstack()

# Sort the data by the total count of loan statuses in descending order
grouped_data = grouped_data.loc[grouped_data.sum(axis=1).sort_values(ascending=False).index]

# Plot the bar chart
grouped_data.plot(kind='bar', stacked=True, figsize=(10, 7))

# Adding labels and title
plt.title('Loan Status by Address State')
plt.xlabel('Address State')
plt.ylabel('Count of Loan Status')
plt.legend(title='Loan Status')

# Display the plot
plt.show()


In [ ]:
# Calculate the total number of loans for each state
total_loans_by_state = comparison_loan_status_addr_state.sum(axis=0)

# Extract the number of defaulted loans for each state
defaulted_loans_by_state = comparison_loan_status_addr_state.loc['Defaulted']

# Calculate the percentage of defaulted loans
defaulted_percentage_by_state = (defaulted_loans_by_state / total_loans_by_state) * 100

# Combine into a DataFrame for easy viewing
defaulted_percentage_df = pd.DataFrame({
    'Total Loans': total_loans_by_state,
    'Defaulted Loans': defaulted_loans_by_state,
    '% Defaulted': defaulted_percentage_by_state
})

# Display the result
defaulted_percentage_df = defaulted_percentage_df.sort_values(by='% Defaulted', ascending=False)
defaulted_percentage_df


#### Key Observations
- **Top States**: The states with the highest loan counts include **CA (California)**, **TX (Texas)**, **NY (New York)**, and **FL (Florida)**. These states exhibit a high volume of loans, likely due to their larger populations and economic activities.
- **Completed Loans**: The **Completed** loan status (blue segment) constitutes a significant portion in most states, suggesting a high rate of loan completion across regions.
- **In Progress Loans**: The **In Progress** loan status (green segment) appears prominently in states with higher loan volumes, indicating ongoing loan activities.
- **Defaulted and Late Loans**: **Defaulted** (orange) and **Late** (red) loans make up smaller portions of the overall loan distribution. However, states with higher loan counts (e.g., CA, TX, NY) also show relatively higher counts in these categories.


Let's keep `addr_state` in our feature set.

## Post EDA Analysis data transformation


Let's call our list function with the new_features selected. Then sense check the results for duplicates or missing data.

In [ ]:
# this calls the split_data_frame function above create three lists to capture the sorting outputs in. 
# These will later be used to pull some graphs to evaluate the data and what possible transformations we've missed.
# Our function parameters are list and the latest version of our data frame in this caes new_features and df_dropped accordingly.
boolean_list, numerical_list, categorical_list = split_data_frame(new_features, df_dropped)

In [ ]:
# Filter columns by datetime dtype
datetime_headers = df_dropped.select_dtypes(include=['datetime64']).columns.tolist()

# Print the list of datetime headers
print("Datetime headers in data frame:", datetime_headers)

In [ ]:
# Removing data time columns as we aren't running a time series.
# List of datetime headers
datetime_headers = df_dropped.select_dtypes(include=['datetime64']).columns.tolist()

# Function to remove datetime headers from a given list
def remove_datetime_headers(feature_list, datetime_headers):
    return [feature for feature in feature_list if feature not in datetime_headers]

# Removing datetime columns from each feature list
boolean_list = remove_datetime_headers(boolean_list, datetime_headers)
numerical_list = remove_datetime_headers(numerical_list, datetime_headers)
categorical_list = remove_datetime_headers(categorical_list, datetime_headers)

# Print updated lists to verify
print("Updated boolean_list:", boolean_list)
print("Updated numerical_list:", numerical_list)
print("Updated categorical_list:", categorical_list)


In [ ]:
def check_for_duplicates(*lists):
    """
    Checks for duplicate headers in each list and prints the duplicates if any are found.

    Parameters:
    ----------
    *lists : list
        Lists of headers to check for duplicates.

    Returns:
    --------
    None
    """
    for i, header_list in enumerate(lists, start=1):
        duplicates = [item for item in set(header_list) if header_list.count(item) > 1]
        if duplicates:
            print(f"Duplicates found in list {i}: {duplicates}")
        else:
            print(f"No duplicates found in list {i}.")

check_for_duplicates(boolean_list, numerical_list, categorical_list)


In [ ]:
def remove_duplicates(*lists):
    """
    Removes duplicates from each list and returns lists with unique headers.

    Parameters:
    ----------
    *lists : list
        Lists of headers to remove duplicates from.

    Returns:
    --------
    unique_lists : list
        A list of lists, each with duplicates removed.
    """
    unique_lists = [list(set(header_list)) for header_list in lists]
    
    return unique_lists

boolean_list, numerical_list, categorical_list = remove_duplicates(boolean_list, numerical_list, categorical_list)

print("Boolean list without duplicates:", boolean_list)
print("Numerical list without duplicates:", numerical_list)
print("Categorical list without duplicates:", categorical_list)


In [ ]:
def check_lists_against_dataframe(dataframe, *lists):
    """
    Checks each provided list for missing and duplicate columns in the DataFrame.

    Parameters:
    ----------
    dataframe : pd.DataFrame
        The DataFrame to check against.
    *lists : list
        Lists of column names to check.

    Returns:
    --------
    None
    """
    # Iterate over each list and check for missing columns and duplicates
    for i, header_list in enumerate(lists, start=1):
        # Convert list to a set to check for missing items and duplicates
        list_set = set(header_list)
        
        # Find columns in the list that are not in the DataFrame
        missing_in_df = list_set - set(dataframe.columns)
        
        # Display results
        print(f"\nList {i} Results:")
        if missing_in_df:
            print(f"Columns missing in DataFrame: {missing_in_df}")
        else:
            print("All columns are present in DataFrame.")

# Run the function with each list
check_lists_against_dataframe(df_dropped, boolean_list, numerical_list, categorical_list)


In [ ]:
df_transformed = df_dropped[new_features].copy()

In [ ]:
# Check if two DataFrames are exactly the same
are_identical = df_dropped[new_features].equals(df_transformed)
print("DataFrames are identical:", are_identical)


In [ ]:
print("Columns in df_dropped but not in df_transformed:", set(df_dropped[new_features].columns) - set(df_transformed.columns))
print("Columns in df_transformed but not in df_dropped:", set(df_transformed.columns) - set(df_dropped.columns))


In [ ]:
df_transformed.head()

In [ ]:
df_transformed[categorical_list].head()

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

def transform_features(dataframe, bool_features, num_features, cat_features):
    """
    Transforms features in the dataframe based on their data type and specified feature lists.

    Parameters:
    ----------
    dataframe : pd.DataFrame
        The DataFrame containing the data to be transformed.
    bool_features : list
        A list of column names representing boolean features.
    num_features : list
        A list of column names representing numerical features.
    cat_features : list
        A list of column names representing categorical features.

    Returns:
    --------
    df_transformed : pd.DataFrame
        The DataFrame with transformed features.
    """
    # Copy the original DataFrame to avoid modifying it
    df_transformed = dataframe.copy()

    # Process boolean features (convert to integers, skip scaling)
    for feature in bool_features:
        if feature in df_transformed.columns:
            print(f"Processing boolean feature: {feature}")
            df_transformed[feature] = df_transformed[feature].astype(bool).astype(int)
        else:
            print(f"Boolean feature '{feature}' not found in DataFrame. Skipping...")

    # Helper function to handle skewness and log transformation
    def handle_skewness(series):
        skewness = series.skew()
        if skewness > 1 or skewness < -1:
            # Apply log transformation to reduce skewness
            feature_min = series.min()
            if feature_min <= 0:
                return np.log1p(series - feature_min + 1)
            else:
                return np.log1p(series)
        return series

    # Process numerical features (apply transformations and scale at the end)
    num_features_to_scale = []
    for feature in num_features:
        if feature in df_transformed.columns:
            print(f"Processing numerical feature: {feature}")
            df_transformed[feature] = df_transformed[feature].fillna(0)  # Impute missing values
            df_transformed[feature] = handle_skewness(df_transformed[feature])  # Apply skewness handling
            num_features_to_scale.append(feature)  # Collect features for batch scaling
        else:
            print(f"Numerical feature '{feature}' not found in DataFrame. Skipping...")

    # Scale all numerical features in a single step
    scaler = StandardScaler()
    df_transformed[num_features_to_scale] = scaler.fit_transform(df_transformed[num_features_to_scale])

    # Process categorical features (One-Hot Encoding, ensure no duplicate encoding later)
    cat_features = [col for col in cat_features if col in df_transformed.columns]
    if cat_features:
        print(f"Processing categorical features: {cat_features}")
        dummies = pd.get_dummies(df_transformed[cat_features], prefix=cat_features)
        df_transformed = pd.concat([df_transformed, dummies], axis=1)
        df_transformed.drop(columns=cat_features, inplace=True)

    print("Transformation complete")
    return df_transformed

# Example usage (ensure boolean_list, numerical_list, and categorical_list are defined)
# df_transformed = transform_features(df_dropped, boolean_list, numerical_list, categorical_list)


In [ ]:
df_transformed = transform_features(df_dropped, boolean_list, numerical_list, categorical_list)

In [ ]:
df_transformed.shape

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
import pandas as pd
from tqdm import tqdm

# Define the VIF calculation function
def calculate_vif(df):
    """
    Calculate VIF (Variance Inflation Factor) for each numeric feature in the given DataFrame.
    
    Parameters:
    ----------
    df : pd.DataFrame
        DataFrame containing the features to check for multicollinearity.
    
    Returns:
    --------
    vif_data : pd.DataFrame
        DataFrame with features and their VIF scores, sorted from highest to lowest.
    """
    # Select only numeric columns, as VIF is applicable to numeric data
    X = df.select_dtypes(include=[float, int]).dropna()
    
    # Initialize the progress bar
    tqdm.pandas(desc="Calculating VIF")

    # Calculate VIF for each feature with progress tracking
    vif_data = pd.DataFrame()
    vif_data["Feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in tqdm(range(X.shape[1]), total=X.shape[1])]

    # Sort VIF data from highest to lowest
    vif_data = vif_data.sort_values(by="VIF", ascending=False).reset_index(drop=True)
    
    # Print features with high VIF scores
    high_vif = vif_data[vif_data["VIF"] > 5]
    print("\nFeatures with high VIF scores (indicating multicollinearity):")
    print(high_vif)
    
    return vif_data


Let's run our VIF on our data frame (df_transformed)

In [ ]:
# Apply VIF calculation to all numeric columns in df_transformed
vif_data_all = calculate_vif(df_transformed)

In [ ]:
# List of columns to remove
columns_to_remove = [
    'mo_sin_old_rev_tl_op_missing_clean_kn'] 

"""
    'num_accts_ever_120_pd_missing_clean_kn',
    'fico_range_high',
    'fico_range_low',
    'out_prncp_inv',
    'out_prncp',
    'loan_amnt',
    'funded_amnt',
    'funded_amnt_inv',
    'tot_cur_bal',
    'total_pymnt',
    'total_pymnt_inv',
    'avg_cur_bal',
    'total_acc',
    'total_bal_il',
    'num_sats',
    'total_bal_ex_mort',
    'num_rev_tl_bal_gt_0',
    'num_actv_rev_tl',
    'tot_hi_cred_lim',
    'total_bc_limit',
    'total_rev_hi_lim',
    'num_op_rev_tl',
    'dti_joint',
    'annual_inc_joint',
    'num_rev_accts',
    'total_rec_prncp',
    'pct_tl_nvr_dlq',
    'acc_open_past_24mths',
    'installment',
    'issue_d_year_kn',
    'bc_util',
    'open_acc',
    'open_act_il',
    'all_util',
    'open_rv_24m',
    'revol_util_kn',
    'revol_bal',
    'bc_open_to_buy',
    'num_bc_sats',
    'max_bal_bc',
    'pub_rec',
    'num_actv_bc_tl',
    'num_bc_tl',
    'num_il_tl',
    'il_util',
    'open_il_24m',
    'open_il_12m',
    'pub_rec_bankruptcies',
    'num_tl_op_past_12m',
    'mths_since_last_record',
    'inq_last_12m_missing_clean_kn',
    'inq_fi_missing_clean_kn'
    
"""

# Removing specified columns from df_transformed
df_transformed = df_transformed.drop(columns=columns_to_remove, errors='ignore')

In [ ]:
# Apply VIF calculation to all numeric columns in df_transformed
vif_data_all = calculate_vif(df_transformed)

### Variance Inflation Factor (VIF) Analysis

VIF analysis helps us detect multicollinearity among features, where high VIF values indicate strong correlations that can destabilize regression models and reduce interpretability. Reducing multicollinearity improves model robustness, decreases overfitting, and ensures reliable coefficient estimates.

#### Why Run VIF?
By identifying features with high VIF, we can remove those causing redundancy, leading to a more stable model. Features with VIF scores over 10 are typically problematic and indicate a strong correlation with other features.

#### Next Steps

Remove High VIF Features: Begin with the feature with the highest VIF score, as it contributes most to multicollinearity.
Recalculate VIF: After each removal, recheck VIF to assess the impact on remaining features.
Repeat as Needed: Continue until all VIF scores are within an acceptable range, typically below 10.

**Interpreting VIF Scores**

#### VIF values are generally interpreted as follows:

- VIF = 1: No correlation with other variables.
- 1 < VIF < 5: Low to moderate correlation.
- VIF > 5: High correlation, potentially problematic.
- VIF > 10: Very high correlation, indicating strong multicollinearity that should be addressed.

In [ ]:
# Filter columns by datetime dtype
df_object = df_transformed.select_dtypes(include=['datetime64']).columns.tolist()

# Print the list of datetime headers
print("Datetime headers in data frame:", df_object)

In [ ]:
# List of columns to remove
columns_to_remove = ['earliest_cr_line', 'issue_d', 'emp_length', 'emp_title', 'home_ownership', 'int_rate', 'loan_status', 'revol_util', 'term', 'zip_code']

# Removing specified columns from df_transformed
df_transformed = df_transformed.drop(columns=columns_to_remove, errors='ignore')

## Pre processing

Troubleshooting the new_features list. During the encoding process I forgot to remove the original columns. This caused string to float conversion errors.

In [ ]:
# Filter columns by dtype
df_object = df_transformed.select_dtypes(include=['object']).columns.tolist()

# Print the list of headers
print("Datetime headers in data frame:", df_object)

In [ ]:
# Elements from datetime_headers that are also in new_features
in_new_features = [item for item in df_object if item in new_features]
# Elements from datetime_headers that are not in new_features
not_in_new_features = [item for item in df_object if item not in new_features]

print("Elements in df that are in new_features:", in_new_features)
print("Elements in df that are NOT in new_features:", not_in_new_features)

In [ ]:
# Filter columns by dtype
datetime_headers = df_transformed.select_dtypes(include=['datetime64']).columns.tolist()

# Print the list of headers
print("Datetime headers in data frame:", datetime_headers)

In [ ]:
# Scanning df_transformed for the value "< 1 year" and identifying columns that contain it

# Check if the value "< 1 year" exists in each column
#matching_columns = df_transformed.apply(lambda col: col.astype(str).str.contains(r"1", na=False)).any()

# Filter to get columns that contain the value "< 1 year"
#columns_with_value = matching_columns[matching_columns].index.tolist()

# Output the result
#print("Columns containing your word:", columns_with_value)

Let's reduce our loan_status_grouped_kn

In [ ]:
df_dropped['loan_status_grouped_kn'].value_counts()

In [ ]:
import pandas as pd

# Assuming your DataFrame is named df_dropped

def group_loan_status(status):
    if status in ("Completed", "In Progress"):
        return "NoDefault"
    elif status in ("Defaulted", "Late"):
        return "Defaulted"
    else:
        return status

df_dropped['loan_status_grouped2_kn'] = df_dropped['loan_status_grouped_kn'].apply(group_loan_status)

# Print the value counts of the new field
print(df_dropped['loan_status_grouped2_kn'].value_counts())

In [ ]:
df_dropped[['loan_status_grouped2_kn','loan_status_grouped_kn']].value_counts()

In [ ]:
# Basic Scikit-learn classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier

# Additional classifiers from external libraries
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

# Single Classification - boolean Y value
# Configured for binary classification
models_single = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Decision Tree': DecisionTreeClassifier(random_state=50),
    #'Random Forest': RandomForestClassifier(random_state=50, n_estimators=100),
    #'Gradient Boosting': GradientBoostingClassifier(random_state=50, n_estimators=100),
    #'SVM': SVC(probability=True, random_state=50),
    #'KNN': KNeighborsClassifier(n_neighbors=5),
    #'Naive Bayes': GaussianNB(),
}

# Multi Classification - Multiple Y values
# Configured for multi-class classification
models_multi = {
    'Logistic Regression': LogisticRegression(max_iter=1000, solver="lbfgs"),
    'Decision Tree': DecisionTreeClassifier(random_state=50),
    'Random Forest': RandomForestClassifier(random_state=50, n_estimators=100),
    #'Gradient Boosting': GradientBoostingClassifier(random_state=50, n_estimators=100),
    #'SVM': SVC(probability=True, random_state=50, decision_function_shape='ovr'),
    #'KNN': KNeighborsClassifier(n_neighbors=5),
    #'Naive Bayes': GaussianNB(),
    #'XGBoost': XGBClassifier(objective="multi:softmax", random_state=50, n_estimators=100),
    #'LightGBM': LGBMClassifier(objective="multiclass", random_state=50, n_estimators=100),
    #'CatBoost': CatBoostClassifier(loss_function="MultiClass", random_state=50, iterations=100),
    #'Neural Network': MLPClassifier(random_state=50, max_iter=300)
}

In [ ]:
# Data Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Define the save path as a constant for the ML results (*.pkl)
SAVE_PATH = "c:\\Users\\kiera\\OneDrive\\Documents\\GitHub\\dsif-git-main-project\\elvtr_main_project\\models\\"

# Assign your dataframes
X = df_transformed  # Use the output from our encoded data
Y = df_dropped['loan_status_grouped2_kn']

# Optional: Encode target variable if necessary
if Y.dtype == 'object' or len(Y.unique()) > 2:
    label_encoder = LabelEncoder()
    Y = label_encoder.fit_transform(Y)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.5, random_state=50
)

# Model selection prompt
def get_model_choice():
    model_type = input("Select model type to run ('single' for models_single or 'multi' for models_multi): ").strip().lower()
    if model_type == 'single':
        print("Selected: Single Classification Model")
        return models_single, None  # No multi-class strategy needed for single
    elif model_type == 'multi':
        multi_class_strategy = input("Select multi-class strategy ('ovo' for one-vs-one or 'ovr' for one-vs-rest): ").strip().lower()
        print(f"Selected: Multi Classification Model with '{multi_class_strategy}' strategy")
        return models_multi, multi_class_strategy
    else:
        print("Invalid selection. Please choose 'single' or 'multi'.")
        return get_model_choice()  # Re-prompt until a valid choice is made

models, multi_class_strategy = get_model_choice()


## Model Definitions

## Evaluation Function

In [ ]:
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier

# Function to wrap model in appropriate multi-class strategy
def get_wrapped_model(model, strategy):
    """
    Wraps the provided model in a multi-class strategy if specified.

    Parameters:
    ----------
    model : estimator object
        The machine learning model to be wrapped in a multi-class strategy.
    strategy : str or None
        Multi-class strategy, either 'ovo' (one-vs-one) or 'ovr' (one-vs-rest).
        If None or unsupported, returns the model unwrapped.

    Returns:
    -------
    estimator object
        Model wrapped in the specified multi-class strategy, or the original model.
    """
    if strategy == 'ovo':
        return OneVsOneClassifier(model)
    elif strategy == 'ovr':
        return OneVsRestClassifier(model)
    elif strategy is None:
        return model  # No wrapping needed for single classification
    else:
        print(f"Warning: Unsupported strategy '{strategy}'. Returning base model.")
        return model  # Fallback to base model for unsupported strategies

In [ ]:
# Function to evaluate binary classification models
def evaluate_model_single(name, model, X_train, X_test, y_train, y_test, save_path=SAVE_PATH):
    print(f"\nTraining {name}...")
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Check if model provides probability estimates
    if hasattr(model, "predict_proba"):
        y_pred_proba = model.predict_proba(X_test)[:, 1]
        roc_auc = roc_auc_score(y_test, y_pred_proba)
        
        # ROC Curve
        fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
        plt.figure()
        plt.plot(fpr, tpr, label=f'{name} (AUC = {roc_auc:.2f})')
        plt.plot([0, 1], [0, 1], 'k--')
        plt.title(f'ROC Curve - {name}')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.legend(loc='lower right')
        plt.grid(True)
        plt.show()

        # Precision-Recall Curve
        precision, recall, _ = precision_recall_curve(y_test, y_pred_proba)
        avg_precision = average_precision_score(y_test, y_pred_proba)
        plt.figure()
        plt.plot(recall, precision, label=f'Average Precision = {avg_precision:.2f}')
        plt.title(f'Precision-Recall Curve - {name}')
        plt.xlabel('Recall')
        plt.ylabel('Precision')
        plt.legend(loc='lower left')
        plt.grid(True)
        plt.show()
    else:
        roc_auc = None

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, zero_division=0)
    recall = recall_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)
    cm = confusion_matrix(y_test, y_pred)

    # Cross-Validation Mean Accuracy
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
    cv_mean_accuracy = cv_scores.mean()

    # Print metrics in desired format
    print(f"Model: {name}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-Score: {f1:.4f}")
    print(f"ROC-AUC: {roc_auc:.4f}" if roc_auc is not None else "ROC-AUC: N/A")
    print(f"Cross-Validation Mean Accuracy: {cv_mean_accuracy:.4f}")
    print("Confusion Matrix:")
    print(cm)

    # Display Confusion Matrix
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title(f'Confusion Matrix - {name}')
    plt.show()

    # Save model and results
    results = {
        'Model': name, 'Accuracy': accuracy, 'Precision': precision,
        'Recall': recall, 'F1-Score': f1, 'ROC-AUC': roc_auc,
        'Cross-Validation Mean Accuracy': cv_mean_accuracy
    }
    if save_path:
        joblib.dump(model, f"{save_path}/{name}_model.pkl")
        joblib.dump(results, f"{save_path}/{name}_results.pkl")
    
    return results


In [ ]:
# Multi-class classification evaluation function
def evaluate_model_multi(name, model, X_train, X_test, y_train, y_test, save_path=SAVE_PATH, multi_class_strategy='ovr'):
    print(f"\nTraining {name}...")
    model = get_wrapped_model(model, multi_class_strategy)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # ROC and PR Curves for each class
    if hasattr(model, "predict_proba"):
        y_pred_proba = model.predict_proba(X_test)
        y_test_binarized = label_binarize(y_test, classes=range(len(set(y_test))))
        n_classes = y_test_binarized.shape[1]

        # ROC Curves for each class
        plt.figure(figsize=(10, 8))
        for i in range(n_classes):
            fpr, tpr, _ = roc_curve(y_test_binarized[:, i], y_pred_proba[:, i])
            roc_auc = roc_auc_score(y_test_binarized[:, i], y_pred_proba[:, i])
            plt.plot(fpr, tpr, label=f'Class {i} (AUC = {roc_auc:.2f})')
        plt.plot([0, 1], [0, 1], 'k--')
        plt.title(f'Multi-Class ROC Curve - {name}')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.legend(loc='lower right')
        plt.grid(True)
        plt.show()

        # PR Curves for each class
        plt.figure(figsize=(10, 8))
        for i in range(n_classes):
            precision, recall, _ = precision_recall_curve(y_test_binarized[:, i], y_pred_proba[:, i])
            avg_precision = average_precision_score(y_test_binarized[:, i], y_pred_proba[:, i])
            plt.plot(recall, precision, label=f'Class {i} (AP = {avg_precision:.2f})')
        plt.xlabel('Recall')
        plt.ylabel('Precision')
        plt.title(f'Multi-Class Precision-Recall Curve - {name}')
        plt.legend(loc='lower left')
        plt.grid(True)
        plt.show()

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
    cm = confusion_matrix(y_test, y_pred)

    # Cross-Validation Mean Accuracy
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
    cv_mean_accuracy = cv_scores.mean()

    # Print metrics in desired format
    print(f"Model: {name}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-Score: {f1:.4f}")
    print(f"ROC-AUC: {roc_auc:.4f}" if roc_auc is not None else "ROC-AUC: N/A")
    print(f"Cross-Validation Mean Accuracy: {cv_mean_accuracy:.4f}")
    print("Confusion Matrix:")
    print(cm)

    # Display Confusion Matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title(f'Confusion Matrix - {name}')
    plt.show()

    # Save model and results
    results = {
        'Model': name, 'Accuracy': accuracy, 'Precision': precision,
        'Recall': recall, 'F1-Score': f1, 'ROC-AUC': roc_auc,
        'Cross-Validation Mean Accuracy': cv_mean_accuracy
    }
    if save_path:
        joblib.dump(model, f"{save_path}/{name}_model.pkl")
        joblib.dump(results, f"{save_path}/{name}_results.pkl")
    
    return results


### Data Leakage troubleshooting

#### Check for predictors

#### Check Train and Test split integrity

In [ ]:
# Check for overlap between train and test indices (should be empty)
train_test_overlap = set(X_train.index).intersection(set(X_test.index))

print("Overlap between train and test sets (should be empty):")
print(train_test_overlap)

#### Verify features are derived from target variable

In [ ]:
# Simple check to see if target-like columns are present in X
target_like_columns = [col for col in X_train.columns if 'loan_status_grouped2_kn' in col or y_train.name in col]

print("Columns potentially derived from target:")
print(target_like_columns)


#### Ensure correct handling of categorical encoding

In [ ]:
from category_encoders import TargetEncoder

# Fit TargetEncoder only on training data
encoder = TargetEncoder()
X_train_encoded = encoder.fit_transform(X_train, y_train)

# Apply to test data without refitting
X_test_encoded = encoder.transform(X_test)

#### Inspect cross validation for leakage

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score

# Example pipeline for cross-validation
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', LogisticRegression())
])

# Apply cross-validation using the pipeline, which prevents data leakage
cv_scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='accuracy')
print("Cross-validation scores:", cv_scores)


## Execute Models

We've extracted, created, cleaned, and encoded our features. Time to run the machine learning models. I've tweaked the code to run several machine learning models since we're focusing on regression type models.

In [ ]:
from tqdm import tqdm

# Evaluating models based on user selection with progress bar
results = []
if models is not None:
    for name, model in tqdm(models.items(), desc="Evaluating models"):
        # Choose the correct evaluation function based on model type
        if multi_class_strategy:
            result = evaluate_model_multi(
                name, model, X_train, X_test, y_train, y_test,
                save_path=SAVE_PATH,
                multi_class_strategy=multi_class_strategy  # Pass strategy to evaluation function
            )
        else:
            result = evaluate_model_single(
                name, model, X_train, X_test, y_train, y_test,
                save_path=SAVE_PATH
            )
            
        results.append(result)
else:
    print("Model evaluation was not performed due to invalid selection.")


## First Model Run Conclusion:

## Summary of Results:

- **Best Recall**: Naive Bayes, though its low precision and high false-positive make it my third choice.
- **Balanced Models**: Random Forest (#1) and Gradient Boosting (#2) offer the best balance between precision, recall, and AUC. I'd suggest working with the features to improve the scoring of these models.

### Detail:

#### 1. Logistic Regression:

The Logistic Regression model achieves a good accuracy and AUC score, indicating some predictive capability. 

However, the low recall indicates it struggles to capture true positive loan defaults.

#### 2. Decision Tree:

The Decision Tree model performs lower than Logistic Regression in terms of AUC.

But it has a better recall, implying it is better at identifying loan defaults compared to Logistic Regression.

#### 3. Random Forest:

Random Forest achieves the highest accuracy and AUC score, but with a lower recall. 

It is more balanced than Logistic Regression (both Precision and Accuracy in the 80s) but scores worse than the Decision Tree in identifying true positives (45 DT compared to 27 RF).

#### 4. Gradient Boosting:

Similar to Random Forest, Gradient Boosting has high accuracy and AUC but lower recall. 

#### 5. K-Nearest Neighbors (KNN):

KNN has lower performance compared to the previous models, particularly struggling with recall and AUC.

We can safely elminate this model.

#### 6. Naive Bayes:

Naive Bayes has a high recall, but very poor accuracy and precision. 

It identifies nearly all defaults, but with many false positives.

## Managing imbalance to improve our model outputs

### SMOTE

We noticed earlier that the data set was largely skewed. Leading to an imbalence in our data set. Let's try and account for this using SMOTE and see if this improves our results.

In [ ]:
from imblearn.over_sampling import SMOTE

# Reduce k_neighbors to handle the small minority class
smote = SMOTE(random_state=50, k_neighbors=3)

# Resample the training data
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

In [ ]:
from tqdm import tqdm

# Evaluating models based on user selection with progress bar
results = []
if models is not None:
    for name, model in tqdm(models.items(), desc="Evaluating models"):
        # Choose the correct evaluation function based on model type
        if multi_class_strategy:
            result = evaluate_model_multi(
                name, model, X_resampled, X_test, y_resampled, y_test,
                save_path=SAVE_PATH,
                multi_class_strategy=multi_class_strategy  # Pass strategy to evaluation function
            )
        else:
            result = evaluate_model_single(
                name, model, X_resampled, X_test, y_resampled, y_test,
                save_path=SAVE_PATH
            )
            
        results.append(result)
else:
    print("Model evaluation was not performed due to invalid selection.")


## Second Model Run Conclusion:

## Conclusion:

Overall reduced precision resulted in drastically better ROC results at the sacrifice of precision (avg. 0.20 pts).

After applying SMOTE, the performance of most models improved in terms of recall, particularly:
- Random Forest: improvements overall besides Precision which dropped a few points. ROC score improved from 0.27 to 0.43 equal to a 60% improvement.
- Gradient Boosting: improvements overall besides Precision, Accuracy, and as a consequence ROC-AUC. ROC score improved from 0.28 to 0.49 equal to a 75% improvement.
and Logistic Regression: improvements overall but suffered by a decrease in accuracy, and precision. This model improved its ROC score the most going from 0.09 to 0.63 after SMOTE equal to a 600% improvement.

### Recommendations:
1. **Random Forest** remains a top choice due to its overall balanced performance, strong ROC-AUC (0.8412), and improved recall after SMOTE.
2. **Gradient Boosting** is another strong option, with a good F1-score and high ROC-AUC (0.8155), performing well across metrics.

These two models are the ones I'd leverage be the primary algorithms selected for predicting loan defaults after applying SMOTE.

## The cost of being wrong

In attempt to understand the cost of error I've used a sample number of 1000 predictions. The question I'm asking myself is the following:

How much is the cost of the time spent researching, execution, and closing false negatives, and false positives?

I'll use Precision and Recall to calculate this. We'll also substitue a few numbers to simulate cost in minutes.

We have the following information:

- **Gradient Boosting:**
    - Precsion (after SMOTE) 0.5523
    - Recall (after SMOTE) 0.4991

- **Random Forest:**
    - Precsion (after SMOTE) 0.6833
    - Recall (after SMOTE) 0.4325

Expected number of defaults = Sum of defaults / Total data set =  12431 / 63689 = 0.1951 * 100 = 20%

Using 1000 applicants or current loans as a base figure.

1000 * 0.20 = 800 estimated non defaults, leaving us with 200 actual defaults estimated.

- **Gradient Boosting:**
    - False Positives = Precsion (after SMOTE) = (1 - 0.5523) * 800 =  358
    - False Negatives = Recall (after SMOTE) = (1 - 0.4991) * 200 = 100
    - Total of incorrect predictions 458

- **Random Forest:**
    - False Positives = Precsion (after SMOTE) = (1 - 0.6833) * 800 =  253
    - False Negatives = Recall (after SMOTE) = (1 - 0.4325) * 200 = 114
    - Total of incorrect predictions 367

The cost of for each approach can be calculated by multiplying our results against the cost of a False Positive, and the cost of a False Negative.

Let's assume the cost of a False Positive (revenue loss maybe denying a loan) equates to 2500, and the cost of a False Negtive (loss from a customer defaulting) equates to 5600.

**Gradient Boosting** = (358 * 2500) + (100 * 5600) = 1,455,000
**Random Forest** = (253 * 2500) + (114 * 5600) = 1,270,900

In the above scenario with these very subjective numbers we'd be best opting for the Random Forest which has a lower cost of being wrong.



In [ ]:
def check_infinity(df):
    infinite_list = df.isin([-np.inf, np.inf]).sum()

    if infinite_list.sum() == 0:
        print("No column has infinite values")
    else:
        print("Columns with infinite values:")
        print(infinite_list[infinite_list>0]).sort_values(ascending=False)

check_infinity(df_transformed)

In [ ]:
nan_list = df_transformed.isna().sum()

if nan_list.sum() == 0:
    print("No column has NaN values")
else:
    print("Columns with NaN values (sorted high to low):")
    print(nan_list[nan_list > 0].sort_values(ascending=False))

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# 'loan_default' is our target column
X = df_transformed
y = df_dropped['loan_status_grouped2_kn']
X_columns = X.columns

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Encode the target variable for both training and test sets
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

In [ ]:
rf = RandomForestClassifier(n_estimators=150, random_state=42) # sets the reproducability to 42
rfe = RFE(estimator=rf, n_features_to_select=23, step=134, verbose=3) # here we're selecting 23 feautures and eleminate 3 features with every iteration of the RFE process.

# Fit RFE on training data only to prevent data leakage
rfe.fit(X_train, y_train)
selected_features = X_train.columns[rfe.support_] # here we're creating a new list with the features the RFE process has captured.

We've opted for a random forest classifier, but will evalute select kbest (from sklearn.feature_selection import SelectKBest, f_classif) at a later date.

In [ ]:
# Select the top features
#selected_features = X_train.columns[rfe.support_] # I've opted to keep this for memory purpose.
selected_features_names = list(selected_features)

print("Selected Features by RFE:")
print(f"Index: {selected_features}")
print(f"Column names: {selected_features_names}")

The above list is the selected features based on the randomforestclassifier selected earlier. 

We've selected a total of **23 features** and created two new variables (**selected_features** and **selected_features_names**)

In [ ]:
# print(rfe.support_)

# Create a list of tuples with column name and boolean value
feature_selection_results = list(zip(X_train.columns, rfe.support_))

# outputs only true
#print("Feature Selection Results:")
#for feature, selected in feature_selection_results:
#    print(f"{feature}: {selected}")

# outputs only false
print("Features Not Selected:")
for feature, selected in feature_selection_results:
    if not selected:  # Only print if 'selected' is False
        print(f"{feature}: {selected}")

Here we can see which columns were excluded by the randomForestClassifier. These are indicated by their boolean value True or False.

After running into a few errors I've inserted this point to check our data set lengths are the same. Essenitally flagging RFE setup errors.

In [ ]:
print(len(X_train.columns))  # Number of columns in training data
print(len(rfe.support_))  # Length of the boolean mask from RFE

In [ ]:
print(len(selected_features))
print(len(selected_features_names))

Let's run a new test focusing only on the features selected in the RFE excercise. We're expecting better prcision and accuracy results and will tweak as needed using the RFE feature selection.

In theory, these new features will generage better forecast capability, faster training through fewer features, and easier to interpret and trouble shoot models.

Using our RFE selected features "selected_features" we'll rerun our randomforest classifier.

In [ ]:
X_train_selected = X_train[selected_features]
X_test_selected = X_test[selected_features]

# Train a new Random Forest classifier on the selected features
rf_selected = RandomForestClassifier(n_estimators=150, random_state=42)
rf_selected.fit(X_train_selected, y_train)


Our next steps should now be focused on Evaluating performance, and Hyperparameter tuning. Let's start by verifying the results with the new features.

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score

y_pred_selected = rf_selected.predict(X_test_selected)
y_prob_selected = rf_selected.predict_proba(X_test_selected)[:, 1]

# Print evaluation metrics
accuracy = accuracy_score(y_test, y_pred_selected)
precision = precision_score(y_test, y_pred_selected)
recall = recall_score(y_test, y_pred_selected)
auc = roc_auc_score(y_test, y_prob_selected)

print(f"Accuracy: {accuracy:.2f}, Precision: {precision:.2f}, Recall: {recall:.2f}, AUC: {auc:.2f}")


Great results. Above we can see that our model predicts **86%** of the samples in our test data correctly. Our positive predictions are correct **82%** of the time. 

However, our Recall is still low with a score of **33%**.

Our AUC scoring **85%** illustrating that it is capable of destinguising between positive and negative classes with a high level of certinity.

Let's have a look at these results with the confusion matrix.

In [ ]:
from dsif6utility import model_evaluation_report

model_evaluation_report(X_test[selected_features], y_test, y_pred_selected, y_prob_selected)


Our main concern here is the low recall. The model seems to be too conservative. We have a few options here:

- hypertuning our randomforest
- try new models e.g. gradient boost, logistic regression.

Let's circle back later on this.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt 

target = df_dropped['loan_status_grouped2_kn']
flag_plot = False  # Don't show graphs

for col in selected_features_names:
    # Calculate correlation 
    correlation = df_transformed[[col, target]].corr().iloc[0, 1]
        
    print(f"Correlation between {col} and {target}: {correlation:.2f}")
    
    # Create scatter plot (if flag_plot is True)
    if flag_plot:
        plt.figure(figsize=(6, 4))
        
        # Use X_test_selected here
        sns.scatterplot(x=X_test_selected[col], y=y_test)  

        plt.xlabel(col)
        plt.ylabel(target)
        plt.title(f"Correlation: {correlation:.2f}") 
        plt.show()

Let's take a look at our data.

**int_rate_int_clean_kn** stands out as being a lead contributor in the above results.

We can observe modertate results from (<10pts - due to training size):

**acc_open_past_24mths** implying that having more accounts opened could be condusive of a person or family taking on more debt and consequently more likely to default.

**last_credit_pull_d_month_clean_kn** this implies that there is a possible linkage between timing of credit queries and default risk 

**dti** being our debt to load ratio is supprisingly low.

At this piont it is important to remeber that correlation is not equal to causation and that our data set is skewed.

## SHAP Value Visualisation

Let's use SHAP to gain a better understanding of how our selected_features contribute to the machine learning outputs.

## Model Interpretability and Explainability

Let's convert our data into a numpy array for faster calculation, and parallael calculations.

In [ ]:
import matplotlib.pyplot as plt

importances = rf_selected.feature_importances_
print(type(importances))

Let's print our numpy array.

In [ ]:
importances

Looking at the array there aren't any dominant or high features. The values, for the most part, are between 0.01 and 0.09. This implies that the model is dependant on multiple variables opposed to having a strong bias towards or smaller portion of variables.

This said we can distinguish a handful of metrics contributing to higher weight (scores above 0.065). Let's visualise these against a bar chart to better understand their importance against each other. We'll sort the bar chart from hieghest to lowest.

Plotting these value will help us determine which features to drop (e.g. lower than 0.02).

In [ ]:
# Sort the array in descending order of feature importance
indices = np.argsort(importances)[::-1]

# Ensure that col_labels matches the number of features being plotted
col_labels = [selected_features[i] for i in indices[:X_test[selected_features].shape[1]]]  # Ensure the correct number of labels

# Plot the feature importances
plt.figure(figsize=(10, 6))
plt.title("Feature importances")
plt.bar(range(X_test[selected_features].shape[1]), importances[indices[:X_test[selected_features].shape[1]]], align="center")
plt.xticks(ticks=range(X_test[selected_features].shape[1]), labels=col_labels, rotation=90)
plt.tight_layout()
plt.show()


Perfect, looking at the above diagram we notice that the last two features contribute the lowest to our model and that we could arguabgly remove these without negatively impacting our model.

Let's run a SHAP excercise to understand and visually reference how these features contribute to the overall predictive capabilities.

Let's start by training our date using the selected features we identified earlier.

In [ ]:
import shap

# Sample only 100 rows from X_test for faster computation
X_test_sample = X_test.sample(1000) 

# Compute SHAP values for X_test_sample
rf_explainer = shap.TreeExplainer(rf_selected) # rf_selected is the randonforestclassifier
rf_shap_values = rf_explainer.shap_values(X_test_sample)

# Print the shape of the SHAP values to verify
print(f'SHAP values shape: {rf_shap_values[1].shape}')  # This takes the positive values only.

I've had to reduce the sample due to computing power. In the above case I've kept the sample to 1k. This portion of the code takes just under 20 min. to run.

Here we have positive and negative output which matches our expectations in terms of postive or negative loan default prediction.

Let's load the SHAP javascript code.

In [ ]:
print(f'SHAP values shape: {rf_shap_values[1].shape}')
print(f'X_test_sample shape: {X_test_sample.shape}')

In [ ]:
# Initialize JS visualization for interactive plots
shap.initjs()

Let's now plot our features and the sample against a force plot.

In [ ]:
observation_index = 7

# Step 1 - calculate expected value of positive class
# This is baseline or average prediction of the model for the positive class before any specific features for an observation are considered
expected_value_positive_class = rf_explainer.expected_value[1]
print("expected_value_positive_class:", expected_value_positive_class)

# Step 2 - SHAP plot for a specific observation (e.g., the first row in the test set)
shap.force_plot(expected_value_positive_class
                ,  rf_shap_values[observation_index][:, 1] # shap_values_positive_class
                , X_test.iloc[observation_index])

Here we can see that our base value (**0.1945**) represents the average prediction for a positive class, our output value in this case **0.21**.

The red arrows push our predictions higher, and the blue push them lower. The length of the arrows illustrate their relevent importance across the feature sets i.e. mths_since_last_major_derog_missing_clean_kn contributes the strongest to the a negative prediction. 

In other words, if there aren't recent bad ratings (over a 90-day period), then the model will reduce the likelyhood of a loan default. This cumulated with the other RFE features will determine the individual True or False score per row i.e. highly likely or not likely to default.

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Create a function to update SHAP visualizations based on selected observation
def update_shap_plot(observation_index):
    # Clear the previous output to avoid stacking multiple force plots
    clear_output(wait=True)
    
    # Re-display the slider (since clearing the output also clears the slider)
    display(observation_slider)
    
    feature_names = col_labels
    
    # Display SHAP force plot for the selected observation
    display(shap.force_plot(
            expected_value_positive_class
                , rf_shap_values[observation_index][:, 1] 
                , X_test.iloc[observation_index]
                , feature_names = feature_names
           ))

# Create a slider to select the observation (0 to len(X_test)-1)
observation_slider = widgets.IntSlider(
    value=0, min=0, max=len(X_test_sample)-1, step=1, description='Observation:', continuous_update=True)

# Use the interactive function to update the SHAP plot when the slider value changes
observation_slider.observe(lambda change: update_shap_plot(change['new']), names='value')

expected_value_positive_class = rf_explainer.expected_value[1]
print("expected_value_positive_class:", expected_value_positive_class)

# Display the slider and force plot
display(observation_slider)

# Call update_shap_plot to display the initial SHAP plot
update_shap_plot(0)

In the above code we've reduced the length to match our sample variable.

As a reminder our basline/average is **0.1945** anything above or below this value is considered a True or False default prediction.

If we move our observation slider (or enter a value) e.g. **825** our model outputs a value of **0.23** indicating that this row of data is likely to default, the higher the value the more likely this is to occur. If we move our slider to index **390** we can observe an output score of **0.07** suggesting that this individual is unlikely to default on their loan as this is much lower than our baseline of **0.1945**.

Let's now run our machine learning neural network challenger model with the RFE selected features. Before assigning our x and y I want to check the feature list is accurate and reflective of our RFE results.

In [ ]:
# Assign your dataframes
X_neural_net = df_transformed[selected_features]  # saved file from previous excercise. [selected_features] are the features selected by the RFE excercise using our pre selected features from assingment 2. In hindsight I'd probably ignore those and start from scratch using RFE to validate my findings against the total collection of columns within the enhanced data frame.
y_neural_net = df_transformed['loan_status_grouped2_kn']

# Optional: Encode target variable if necessary
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_neural_net)

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_neural_net, y_encoded, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Neural Network Challenger Models

### Model 1: A simple neural network with 16, 8, and 1 neurons in three layers.

In [ ]:
# Define the model
model1a = Sequential([
    Dense(16, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dense(8, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model1a.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history1 = model1a.fit(X_train_scaled, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Plotting training vs validation performance
plt.figure(figsize=(FIG_WIDTH, FIG_HEIGHT))
plt.plot(history1.history['accuracy'], label='Train Accuracy')
plt.plot(history1.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title('Training vs Validation Accuracy')
plt.show()

### Model 2: A neural network with an additional 32-neuron layer.

In [ ]:
# Define the model
model1b = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dense(16, activation='relu'),
    Dense(8, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model1b.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history1 = model1b.fit(X_train_scaled, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Plotting training vs validation performance
plt.figure(figsize=(FIG_WIDTH, FIG_HEIGHT))
plt.plot(history1.history['accuracy'], label='Train Accuracy')
plt.plot(history1.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title('Training vs Validation Accuracy')
plt.show()

### Model 3: A more complex network with layers of 64, 32, 16, and 8 neurons.


In [ ]:
# Define the model
model1c = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(8, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model1c.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history1 = model1c.fit(X_train_scaled, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Plotting training vs validation performance
plt.figure(figsize=(FIG_WIDTH, FIG_HEIGHT))
plt.plot(history1.history['accuracy'], label='Train Accuracy')
plt.plot(history1.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title('Training vs Validation Accuracy')
plt.show()

### Results of our Basic Neural Network Model

We've ran three basic neural networks to evaluate differnet levels of complexity. Our second model performed the best. **needs more input** we'll use the second nn for future excercises.

- **Training Accuracy**: 0.8590
- **Validation Accuracy**: 0.8417
- **Training Loss**: 0.3470
- **Validation Loss**: 0.3799

**Key Observations:**

**Accuracy:** Both the training and validation accuracy are important to measure how well the model is performing on both the training data and unseen data.

The third model has the highest training accuracy at 85.89%, followed by the second model at 85.01%, and the first model at 83.77%.
For validation accuracy, the second and third models are tied at 84.49%, which is higher than the first model's 83.33%.
Validation Loss: This metric indicates how well the model generalizes to unseen data, where a lower value is better.

The second model has the lowest validation loss at 0.3781, followed by the third model at 0.3813, and the first model has the highest validation loss at 0.3914.

**Conclusion:**

The second model seems to perform the best, given that it has:

- High validation accuracy (84.49%).
- The lowest validation loss (0.3781), indicating the best generalization to unseen data.

While the third model has a slightly higher training accuracy, the minimal difference in validation accuracy and higher validation loss compared to the second model suggests the second model is better optimized and generalizes better.

## Regularisation Techniques

##  **Word of warning, I've played around with model parameters. To save you the hassle Model2b performed best**

Apolgies in advance for the hours of life I've taken away from you reading this.

#### Brief recap (better summary towards the end)

The introduction of dropout, early stopping, and L2 regularisation demonstrated improvements in the neural network models' ability to generalise, with Models 2b and 2e outperforming the baseline ensemble models. Despite these gains, recall remains a challenge, highlighting the need for continued work on class imbalance and model sensitivity. Moving forward, these insights provide a solid foundation for enhancing model robustness and achieving better predictive outcomes for loan default classification.

![image.png](attachment:image.png)

### Model 2: Adding Dropout and Early Stopping

To prevent overfitting we'll use dropout and early stopping techniques.

In [ ]:
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Define the model with Dropout layers
model2a = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.5),  # Dropout layer
    Dense(16, activation='relu'),
    Dropout(0.5),
    Dense(8, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model2a.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', 'Precision', 'Recall', 'AUC'])

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

# Train the model with early stopping
history2 = model2a.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

# Plotting training vs validation performance
plt.figure(figsize=(FIG_WIDTH, FIG_HEIGHT))
plt.plot(history2.history['accuracy'], label='Train Accuracy')
plt.plot(history2.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title('Training vs Validation Accuracy (Early Stopping)')
plt.show()


In [ ]:
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Define the model with Dropout layers
model2b = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.2),  # Dropout layer
    Dense(16, activation='relu'),
    Dropout(0.2),
    Dense(8, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

# Compile the model
model2b.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', 'Precision', 'Recall', 'AUC'])

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

# Train the model with early stopping
history2 = model2b.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

# Plotting training vs validation performance
plt.figure(figsize=(FIG_WIDTH, FIG_HEIGHT))
plt.plot(history2.history['accuracy'], label='Train Accuracy')
plt.plot(history2.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title('Training vs Validation Accuracy (Early Stopping)')
plt.show()


In [ ]:
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Define the model with Dropout layers
model2c = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.5),  # Dropout layer
    Dense(16, activation='relu'),
    Dropout(0.5),
    Dense(8, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model2c.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', 'Precision', 'Recall', 'AUC'])

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model with early stopping
history2 = model2c.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

# Plotting training vs validation performance
plt.figure(figsize=(FIG_WIDTH, FIG_HEIGHT))
plt.plot(history2.history['accuracy'], label='Train Accuracy')
plt.plot(history2.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title('Training vs Validation Accuracy (Early Stopping)')
plt.show()


In [ ]:
from tensorflow.keras.layers import BatchNormalization

model2d = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(32, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(16, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model2d.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', 'Precision', 'Recall', 'AUC'])

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

# Train the model with early stopping
history2 = model2d.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

# Plotting training vs validation performance
plt.figure(figsize=(FIG_WIDTH, FIG_HEIGHT))
plt.plot(history2.history['accuracy'], label='Train Accuracy')
plt.plot(history2.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title('Training vs Validation Accuracy (Early Stopping)')
plt.show()


In [ ]:
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Define the model with Dropout layers
model2e = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.2),  # Dropout layer
    Dense(16, activation='relu'),
    Dropout(0.2),
    Dense(8, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model2e.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', 'Precision', 'Recall', 'AUC'])

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

# Train the model with early stopping
history2 = model2e.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

# Plotting training vs validation performance
plt.figure(figsize=(FIG_WIDTH, FIG_HEIGHT))
plt.plot(history2.history['accuracy'], label='Train Accuracy')
plt.plot(history2.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title('Training vs Validation Accuracy (Early Stopping)')
plt.show()


### Neural Network with Dropout and Early Stopping
- **Training Accuracy**: 0.8088
- **Validation Accuracy**: 0.8080
- **Training Loss**: 0.4380
- **Validation Loss**: 0.4267
- **ROC-AUC**: 0.76

**Training and Validation Observations**:
- Early stopping helped mitigate overfitting, keeping training and validation accuracy close.
- However, the ROC-AUC dropped to 0.76, lower than Gradient Boosting and Random Forest.


### Regularization with L2

Add L2 regularization to penalize large weights.

In [ ]:
from tensorflow.keras.regularizers import l2

# Define model with L2 regularization
model3a = Sequential([
    Dense(16, activation='relu', kernel_regularizer=l2(0.01), input_shape=(X_train_scaled.shape[1],)),
    Dense(8, activation='relu', kernel_regularizer=l2(0.01)),
    Dense(1, activation='sigmoid')
])

# Compile the model
model3a.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', 'Precision', 'Recall', 'AUC'])

# Train the model
history3 = model3a.fit(X_train_scaled, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Plotting training vs validation performance
plt.figure(figsize=(FIG_WIDTH, FIG_HEIGHT))
plt.plot(history3.history['accuracy'], label='Train Accuracy')
plt.plot(history3.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title('Training vs Validation Accuracy (L2 Regularization)')
plt.show()


In [ ]:
from tensorflow.keras.regularizers import l2

model3b = Sequential([
    Dense(64, activation='relu', kernel_regularizer=l2(0.001), input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.3),
    Dense(32, activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.3),
    Dense(16, activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

# Compile the model
model3b.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', 'Precision', 'Recall', 'AUC'])

# Train the model
history3 = model3b.fit(X_train_scaled, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Plotting training vs validation performance
plt.figure(figsize=(FIG_WIDTH, FIG_HEIGHT))
plt.plot(history3.history['accuracy'], label='Train Accuracy')
plt.plot(history3.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title('Training vs Validation Accuracy (L2 Regularization)')
plt.show()

### Neural Network with L2 Regularization
- **Training Accuracy**: 0.8117
- **Validation Accuracy**: 0.8130
- **Training Loss**: 0.4380
- **Validation Loss**: 0.4417
- **ROC-AUC**: 0.77

**Training and Validation Observations**:
- L2 Regularization kept training and validation accuracies well-matched.
- ROC-AUC was 0.77, which is also lower compared to Random Forest and Gradient Boosting.

## Overall Comparison
- **Accuracy**: The Random Forest model provided the best accuracy (0.8506), followed by Gradient Boosting and the Basic Neural Network model.
- **ROC-AUC**: The Random Forest model achieved the highest ROC-AUC (0.8412), followed by Gradient Boosting (0.8155). The basic neural network model came close (0.82).
- **Regularization Impact**: Adding dropout with early stopping and L2 regularization helped prevent overfitting but decreased ROC-AUC compared to the Gradient Boosting and Random Forest models.
- **Validation**: Validation accuracy and training accuracy closely match for the regularized models, indicating reduced overfitting.

## Evaluate on Test Set

Calculate metrics for each model on the test data ready to plot in a ROC Curve.

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score

# Helper function to evaluate model on test data
def evaluate_model(model, X_test, y_test):
    y_pred_probs = model.predict(X_test).ravel()
    y_pred = (y_pred_probs > 0.5).astype(int)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred_probs)
    
    return accuracy, precision, recall, auc

# Evaluate models
models = [model1a, model1b, model1c, model2a, model2b, model2c, model2d, model2e, model3a, model3b]
model_names = ["Model1a - Basic NN", "Model1b - Basic NN", "Model1c - Basic NN", "Model2a - Dropout + Early Stopping", "Model2b - Dropout + Early Stopping", "Model2c - Dropout + Early Stopping", "Model2d - Dropout + Early Stopping", "Model2e - Dropout + Early Stopping", "Model3a - L2 Regularization", "Model3b - L2 Regularization"]

for name, model in zip(model_names, models):
    accuracy, precision, recall, auc = evaluate_model(model, X_test_scaled, y_test)
    print(f"{name} - Accuracy: {accuracy:.2f}, Precision: {precision:.2f}, Recall: {recall:.2f}, AUC: {auc:.2f}")


## Plot ROC Curves

Plot the ROC curves for all models to compare their ability to distinguish between positive and negative classes.

In [ ]:
from sklearn.metrics import roc_curve

plt.figure(figsize=(FIG_WIDTH, FIG_HEIGHT))

for name, model in zip(model_names, models):
    y_pred_probs = model.predict(X_test_scaled).ravel()
    fpr, tpr, _ = roc_curve(y_test, y_pred_probs)
    plt.plot(fpr, tpr, label=f'{name} (AUC = {roc_auc_score(y_test, y_pred_probs):.2f})')

plt.plot([0, 1], [0, 1], 'k--', label='Random Guess')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves')
plt.legend()
plt.show()


### Summary of Model Performance - ROC Curves

The plot above shows the **ROC curves** for all models evaluated on the RFE feature selected data frame.

**AUC (Area Under the Curve)** scores provided for each model wherby the best performing modesl are Model2b and Model2e both scoring 0.84:

1. **Model1a, Model1b, Model1c - Basic Neural Networks**:
   - These models achieved **AUC scores** of **0.82 to 0.83**, showing consistent performance across variations. They performed well in distinguishing between positive and negative classes, with relatively high TPR (True Positive Rate) and low FPR (False Positive Rate).

2. **Model2a to Model2e - Dropout + Early Stopping**:
   - These models varied in performance, with **AUC scores ranging from 0.73 to 0.84**. The best-performing Dropout + Early Stopping model (Model2b and Model2e) both achieved an **AUC of 0.84**, indicating strong predictive power.

3. **Model3a and Model3b - L2 Regularization**:
   - The **L2 Regularization** models had **AUC scores of 0.77 and 0.79**. These models were generally more consistent than some of the dropout variants, showing moderate performance compared to the best-performing Dropout + Early Stopping models.

### Key Observations
- **Dropout with Early Stopping** (Model2b and Model2e) produced the best results among all models, achieving AUC scores of **0.84**.
- **Basic Neural Networks** performed consistently well, with **AUC values between 0.82 and 0.83**.
- **L2 Regularisation** models showed stable but slightly lower performance compared to the other methods. I've dragged this on longer than I should and will explore the possible tweaks for L2 regularisation on my own time.

Overall, **Dropout with Early Stopping** appears to be the most effective strategy in this case, providing a good balance between generalization and avoiding overfitting. Future optimization could focus on fine-tuning dropout rates and the regularization parameter to further improve model performance.

If I were to chose one at this point it would be Model2b which has the higher recall from the two models.


Let's load our previous notebook results and compare them against the neural network.

The code to export ML resulsta was added to the assignment 2 code after submission.

In [ ]:
import joblib

# Load pre-trained models
gradient_boosting_model = joblib.load('models/smote/Gradient_Boosting_model.pkl')
random_forest_model = joblib.load('models/smote/Random_Forest_model.pkl')

# Load the saved results
gradient_boosting_results = joblib.load('models/smote/Gradient_Boosting_results.pkl')
random_forest_results = joblib.load('models/smote/Random_Forest_results.pkl')

print(gradient_boosting_results)
print(random_forest_results)

Mindful the previous models were trained on 54 features, and more rows of data. I managed to upscale and compare but wasn't sure if the results can be trusted and decided to remove them in the end.

## Conclusion

In this assignment, we developed several neural network models to serve as challengers to the baseline models from Assignment 2, which included Random Forest and Gradient Boosting. By incorporating various regularisation techniques, such as dropout, early stopping, and L2 regularisation, we aimed to improve model generalisation, reduce overfitting, and increase predictive performance.

## Summary of Results:

The basic neural network models (Models 1a, 1b, 1c) demonstrated consistent accuracy and ROC-AUC scores, with Model 1b performing slightly better (AUC = 0.83).
Dropout with Early Stopping proved to be an effective regularisation technique. Models 2b and 2e achieved the best AUC scores (0.84), outperforming the Random Forest baseline (AUC = 0.8412) and Gradient Boosting (AUC = 0.8155).
L2 Regularisation models (Models 3a and 3b) provided moderate improvement in training stability, but their overall AUC scores (0.77 and 0.79) were slightly lower than other models.
Validation accuracy was generally consistent with training accuracy for the regularised models, indicating reduced overfitting.

## Key Observations:

**Precision and Recall Trade-off:** While the models showed strong precision and accuracy, recall remained relatively low across all models. This suggests that, although the models are confident in their predictions, they struggle with correctly identifying positive instances (loan defaults).

Models 2b and 2e emerged as the most effective challenger models, offering a good balance between precision, recall, and AUC, with strong generalisation.
The ROC curves illustrated that Models 2b and 2e performed the best in distinguishing between positive and negative classes, showing improved predictive power compared to other models.

## Future Improvements:

**Class Imbalance:** The low recall suggests that the model may be biased towards the majority class. This is mostly due to the skewed data but also due to the fact that the hardware limited running the entire dataset.

**Ensemble Methods:** Combining multiple models (e.g., ensembling neural networks with Gradient Boosting) may yield a more robust solution. I haven't figured out how to make this work efficiently just yet and will try to do this in the next assignment.

**Feature Engineering:** Overall I'm happy with the feature selection but believe that it is possible to leverage additional data points, maybe increase our feature count from 23 to X.